In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 1. 한식만

In [6]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.filterwarnings("ignore")

DATA_PATH = "data/수원시 한식 데이터백업.csv"
df = pd.read_csv(DATA_PATH)

# ===== (1) 날짜 컬럼 찾기/통일 =====
date_col = "DATE" if "DATE" in df.columns else ("TA_YMD" if "TA_YMD" in df.columns else None)
if date_col is None:
    raise ValueError("날짜 컬럼이 DATE 또는 TA_YMD로 존재해야 합니다.")

df[date_col] = pd.to_datetime(df[date_col])
df = df.sort_values(date_col).reset_index(drop=True)

TARGET = "AMT"
if TARGET not in df.columns:
    raise ValueError(f"타겟 컬럼 {TARGET} 이(가) 없습니다. 컬럼명 확인 필요.")

# ===== (2) DONG 문자열 처리 =====
cat_cols = []
if "DONG" in df.columns:
    df["DONG"] = df["DONG"].astype("category")
    cat_cols.append("DONG")

# ===== (3) 피처 엔지니어링 =====
def make_features(data):
    d = data.copy()

    # 날짜 파생 (datetime 제거 목적)
    d["year"] = d[date_col].dt.year
    d["month"] = d[date_col].dt.month
    d["day"] = d[date_col].dt.day
    d["dow"] = d[date_col].dt.weekday
    d["weekend"] = (d["dow"] >= 5).astype(int)

    # Lag / Rolling (AMT 기반)
    for lag in [1, 7, 14]:
        d[f"lag_{lag}"] = d[TARGET].shift(lag)

    for win in [7, 14]:
        d[f"roll_mean_{win}"] = d[TARGET].shift(1).rolling(win).mean()
        d[f"roll_std_{win}"] = d[TARGET].shift(1).rolling(win).std()

    return d

df = make_features(df)
df = df.dropna().reset_index(drop=True)

# ===== (4) 학습용 X/y 구성 =====
drop_cols = [TARGET, date_col]
FEATURES = [c for c in df.columns if c not in drop_cols]

X = df[FEATURES].copy()
y = df[TARGET].copy()

# LightGBM이 category dtype을 처리할 수 있도록 유지
# (단, 혹시 object 남아있으면 category로 바꿔줌)
for c in X.columns:
    if X[c].dtype == "object":
        X[c] = X[c].astype("category")
        if c not in cat_cols:
            cat_cols.append(c)

# ===== (5) TimeSeries CV =====
tscv = TimeSeriesSplit(n_splits=4)
mae_scores = []

for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = lgb.LGBMRegressor(
        objective="regression_l1",
        n_estimators=2000,
        learning_rate=0.03,
        num_leaves=64,
        min_child_samples=30,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.3,
        reg_lambda=0.3,
        n_jobs=-1,
        random_state=42
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="l1",
        categorical_feature=cat_cols if len(cat_cols) > 0 else "auto",
        callbacks=[lgb.early_stopping(200, verbose=False)]
    )

    pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, pred)
    mae_scores.append(mae)
    print(f"Fold {fold+1} MAE: {mae:,.0f}")

print("\n====================")
print(f"CV 평균 MAE: {np.mean(mae_scores):,.0f}")
print("====================")

# ===== (6) 최종 모델 =====
final_model = lgb.LGBMRegressor(
    objective="regression_l1",
    n_estimators=int(np.mean([model.best_iteration_ for _ in range(1)]) or 1500),
    learning_rate=0.03,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.3,
    reg_lambda=0.3,
    n_jobs=-1,
    random_state=42
)
final_model.fit(X, y, categorical_feature=cat_cols if len(cat_cols) > 0 else "auto")

print("최종 모델 학습 완료")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002928 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2711
[LightGBM] [Info] Number of data points in the train set: 88705, number of used features: 15
[LightGBM] [Info] Start training from score 2080467.000000
Fold 1 MAE: 126,532
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006031 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2753
[LightGBM] [Info] Number of data points in the train set: 177406, number of used features: 15
[LightGBM] [Info] Start training from score 2119460.000000
Fold 2 MAE: 318,519
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005529 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2790
[Li

# 2. 배달, 한식, 저장

In [8]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
import joblib
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

# =========================
# 경로/설정
# =========================
HAN_PATH = "data/수원시 한식 데이터백업.csv"
DELIVERY_PATH = "data/수원시 배달 데이터백업.csv"

OUT_DIR = "data/models"
os.makedirs(OUT_DIR, exist_ok=True)

DATE_COL = "TA_YMD"
TARGET = "AMT"

# =========================
# 유틸: 날씨 컬럼 자동 탐지 (temp/rain)
# =========================
def find_weather_cols(df, mode: str):
    cols = list(df.columns)
    low = {c: str(c).lower() for c in cols}

    if mode == "temp":
        keys = ["temp", "tavg", "tmean", "기온", "평균기온", "최고기온", "최저기온"]
    elif mode == "rain":
        keys = ["rain", "precip", "prcp", "강수", "강수량", "강우", "mm"]
    else:
        raise ValueError("mode must be 'temp' or 'rain'")

    picked = []
    for c in cols:
        s = low[c]
        if any(k in s for k in keys):
            picked.append(c)

    # 숫자형만 유지
    picked = [c for c in picked if pd.api.types.is_numeric_dtype(df[c])]
    return picked

# =========================
# 피처 생성
#  - 날짜 파생(연/월/일/요일 등)
#  - 동(DONG) 있으면 category로 사용
#  - lag/rolling (동별이면 groupby 적용)
# =========================
def make_features(df: pd.DataFrame, group_col: str | None = "DONG"):
    d = df.copy()

    if DATE_COL not in d.columns:
        raise ValueError(f"날짜 컬럼 {DATE_COL} 없음")
    if TARGET not in d.columns:
        raise ValueError(f"타겟 컬럼 {TARGET} 없음")

    d[DATE_COL] = pd.to_datetime(d[DATE_COL])
    d = d.sort_values(DATE_COL).reset_index(drop=True)

    # 날짜 파생 (datetime dtype 제거 목적)
    d["year"] = d[DATE_COL].dt.year
    d["month"] = d[DATE_COL].dt.month
    d["day"] = d[DATE_COL].dt.day
    d["dow"] = d[DATE_COL].dt.weekday
    d["weekend"] = (d["dow"] >= 5).astype(int)

    cat_cols = []
    if group_col and group_col in d.columns:
        d[group_col] = d[group_col].astype("category")
        cat_cols.append(group_col)
    else:
        group_col = None

    # Lag/Rolling
    def add_lag_roll(x):
        for lag in [1, 7, 14]:
            x[f"lag_{lag}"] = x[TARGET].shift(lag)
        for win in [7, 14]:
            x[f"roll_mean_{win}"] = x[TARGET].shift(1).rolling(win).mean()
            x[f"roll_std_{win}"] = x[TARGET].shift(1).rolling(win).std()
        return x

    if group_col:
        d = d.groupby(group_col, group_keys=False).apply(add_lag_roll)
    else:
        d = add_lag_roll(d)

    # object 남아있으면 category로 (LightGBM dtype 에러 방지)
    for c in d.columns:
        if d[c].dtype == "object" and c != TARGET:
            d[c] = d[c].astype("category")
            if c not in cat_cols:
                cat_cols.append(c)

    return d, cat_cols, group_col

# =========================
# 학습 + 최고 성능 모델 저장
#  - TS CV 4 folds
#  - 각 fold의 best_iteration_ 평균으로 최종 모델 재학습
#  - CV 평균 MAE가 더 낮으면 파일 덮어쓰기
# =========================
def train_and_save_best(df: pd.DataFrame, name: str, mode: str):
    """
    name: 저장 파일 prefix (예: 'han', 'delivery')
    mode: 'temp' or 'rain' (컬럼 자동 탐지용)
    """
    weather_cols = find_weather_cols(df, mode=mode)

    d, cat_cols, group_col = make_features(df, group_col="DONG")

    # 학습 피처: 타겟/날짜 제외 전부 (날씨 컬럼이 포함되어 있으면 자동 포함)
    drop_cols = [TARGET, DATE_COL]
    feature_cols = [c for c in d.columns if c not in drop_cols]

    # NA 제거
    X = d[feature_cols].copy()
    y = d[TARGET].copy()
    mask = ~X.isna().any(axis=1) & ~y.isna()
    X = X[mask].reset_index(drop=True)
    y = y[mask].reset_index(drop=True)

    # TS CV
    tscv = TimeSeriesSplit(n_splits=4)
    maes, best_iters = [], []

    params = dict(
        objective="regression_l1",
        n_estimators=3500,          # early stopping으로 자동 컷
        learning_rate=0.03,
        num_leaves=64,
        min_child_samples=30,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.3,
        reg_lambda=0.3,
        n_jobs=-1,
        random_state=42
    )

    print(f"\n===== [{name}] 학습 시작 (mode={mode}) =====")
    if len(weather_cols) == 0:
        print(f"⚠️ [{name}] {mode} 관련 날씨 컬럼 자동 탐지 실패(그래도 학습은 진행).")
    else:
        print(f"[{name}] 핵심 날씨 컬럼 후보: {weather_cols}")

    for fold, (tr, va) in enumerate(tscv.split(X), start=1):
        model = lgb.LGBMRegressor(**params)
        model.fit(
            X.iloc[tr], y.iloc[tr],
            eval_set=[(X.iloc[va], y.iloc[va])],
            eval_metric="l1",
            categorical_feature=cat_cols if len(cat_cols) else "auto",
            callbacks=[lgb.early_stopping(200, verbose=False)]
        )
        pred = model.predict(X.iloc[va])
        mae = mean_absolute_error(y.iloc[va], pred)
        maes.append(mae)
        best_iters.append(model.best_iteration_)
        print(f"[{name}] Fold {fold} MAE: {mae:,.0f}  (best_iter={model.best_iteration_})")

    cv_mae = float(np.mean(maes))
    final_estimators = int(np.clip(np.mean(best_iters), 300, 3500))
    print(f"[{name}] CV 평균 MAE: {cv_mae:,.0f} / 최종 n_estimators={final_estimators}")

    # 최종 모델 재학습
    final_model = lgb.LGBMRegressor(**{**params, "n_estimators": final_estimators})
    final_model.fit(X, y, categorical_feature=cat_cols if len(cat_cols) else "auto")

    # 저장 경로
    model_path = os.path.join(OUT_DIR, f"{name}_best_model.pkl")
    meta_path  = os.path.join(OUT_DIR, f"{name}_best_meta.pkl")

    # 기존 저장 모델이 있으면 성능 비교 후 더 좋을 때만 덮어쓰기
    should_save = True
    if os.path.exists(meta_path):
        try:
            old_meta = joblib.load(meta_path)
            old_mae = float(old_meta.get("cv_mae", np.inf))
            if cv_mae >= old_mae:
                should_save = False
                print(f"[{name}] 기존 모델이 더 좋거나 동일함: old_CV_MAE={old_mae:,.0f} <= new_CV_MAE={cv_mae:,.0f}")
        except Exception:
            # 메타가 깨져있으면 새로 저장
            should_save = True

    if should_save:
        joblib.dump(final_model, model_path)
        meta = {
            "cv_mae": cv_mae,
            "date_col": DATE_COL,
            "target_col": TARGET,
            "feature_cols": feature_cols,
            "cat_cols": cat_cols,
            "group_col": group_col,
            "mode": mode,
            "weather_cols_detected": weather_cols,
            "n_estimators": final_estimators,
        }
        joblib.dump(meta, meta_path)
        print(f"✅ [{name}] 최고 성능 모델 저장 완료")
        print(f" - model: {model_path}")
        print(f" - meta : {meta_path}")

    return final_model, cv_mae

# =========================
# 실행
# =========================
han_df = pd.read_csv(HAN_PATH)
del_df = pd.read_csv(DELIVERY_PATH)

# 한식 = 기온 중심
han_model, han_cv_mae = train_and_save_best(han_df, name="han", mode="temp")

# 배달 = 강수 중심
del_model, del_cv_mae = train_and_save_best(del_df, name="delivery", mode="rain")

print("\n===== 전체 완료 =====")
print(f"한식 CV MAE: {han_cv_mae:,.0f}")
print(f"배달 CV MAE: {del_cv_mae:,.0f}")



===== [han] 학습 시작 (mode=temp) =====
[han] 핵심 날씨 컬럼 후보: ['TEMP']
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003299 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2711
[LightGBM] [Info] Number of data points in the train set: 88583, number of used features: 15
[LightGBM] [Info] Start training from score 2083101.000000
[han] Fold 1 MAE: 121,904  (best_iter=3499)
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005547 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2753
[LightGBM] [Info] Number of data points in the train set: 177164, number of used features: 15
[LightGBM] [Info] Start training from score 2120076.500000
[han] Fold 2 MAE: 297,366  (best_iter=3500)
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010703 seconds.
You can set `force_col_wise=true` to remove th

✅ [delivery] 최고 성능 모델 저장 완료
 - model: data/models\delivery_best_model.pkl
 - meta : data/models\delivery_best_meta.pkl

===== 전체 완료 =====
한식 CV MAE: 204,039
배달 CV MAE: 6,889


In [11]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# =====================
# 경로
# =====================
HAN_MODEL_PATH = r"data/models/han_best_model.pkl"
HAN_META_PATH  = r"data/models/han_best_meta.pkl"
HAN_DATA_PATH  = r"data/수원시 한식 데이터백업.csv"   # 네 환경 경로로 맞춰

DEL_MODEL_PATH = r"data/models/delivery_best_model.pkl"
DEL_META_PATH  = r"data/models/delivery_best_meta.pkl"
DEL_DATA_PATH  = r"data/수원시 배달 데이터백업.csv"  # 네 환경 경로로 맞춰

# =====================
# 공통: 피처 생성 (학습 코드와 동일해야 함)
# =====================
DATE_COL = "TA_YMD"
TARGET = "AMT"

def make_features(df: pd.DataFrame, group_col: str | None = "DONG"):
    d = df.copy()
    d[DATE_COL] = pd.to_datetime(d[DATE_COL])
    d = d.sort_values(DATE_COL).reset_index(drop=True)

    d["year"] = d[DATE_COL].dt.year
    d["month"] = d[DATE_COL].dt.month
    d["day"] = d[DATE_COL].dt.day
    d["dow"] = d[DATE_COL].dt.weekday
    d["weekend"] = (d["dow"] >= 5).astype(int)

    cat_cols = []
    if group_col and group_col in d.columns:
        d[group_col] = d[group_col].astype("category")
        cat_cols.append(group_col)
    else:
        group_col = None

    def add_lag_roll(x):
        for lag in [1, 7, 14]:
            x[f"lag_{lag}"] = x[TARGET].shift(lag)
        for win in [7, 14]:
            x[f"roll_mean_{win}"] = x[TARGET].shift(1).rolling(win).mean()
            x[f"roll_std_{win}"] = x[TARGET].shift(1).rolling(win).std()
        return x

    if group_col:
        d = d.groupby(group_col, group_keys=False).apply(add_lag_roll)
    else:
        d = add_lag_roll(d)

    # object -> category (LightGBM dtype 방지)
    for c in d.columns:
        if d[c].dtype == "object" and c not in [TARGET]:
            d[c] = d[c].astype("category")
            if c not in cat_cols:
                cat_cols.append(c)

    return d

def wmape(y_true, y_pred, eps=1e-9):
    denom = np.sum(np.abs(y_true)) + eps
    return np.sum(np.abs(y_true - y_pred)) / denom

# =====================
# 평가 함수
# =====================
def evaluate_saved_model(model_path, meta_path, data_path, holdout_ratio=0.2, print_by_dong=True):
    model = joblib.load(model_path)
    meta = joblib.load(meta_path)

    feature_cols = meta["feature_cols"]
    cat_cols = meta.get("cat_cols", [])
    group_col = meta.get("group_col", None)

    df = pd.read_csv(data_path)
    d = make_features(df, group_col=group_col)

    # 학습 때처럼 NA 제거
    X = d[feature_cols].copy()
    y = d[TARGET].copy()
    mask = ~X.isna().any(axis=1) & ~y.isna()
    X = X[mask].reset_index(drop=True)
    y = y[mask].reset_index(drop=True)

    # 시계열 holdout: 뒤쪽 20%를 test로
    n = len(X)
    split = int(n * (1 - holdout_ratio))
    X_train, X_test = X.iloc[:split], X.iloc[split:]
    y_train, y_test = y.iloc[:split], y.iloc[split:]

    # 예측
    pred = model.predict(X_test)

    # 지표
    mae = mean_absolute_error(y_test, pred)
    rmse = mean_squared_error(y_test, pred, squared=False)
    r2 = r2_score(y_test, pred)
    w = wmape(y_test.values, pred)

    print(f"\n==== 평가: {model_path} ====")
    print(f"Samples: train={len(X_train):,} / test={len(X_test):,}")
    print(f"MAE  : {mae:,.0f}")
    print(f"RMSE : {rmse:,.0f}")
    print(f"R2   : {r2:.4f}")
    print(f"WMAPE: {w:.4%}")

        # (선택) 동별 MAE
    if print_by_dong and group_col and group_col in d.columns:
        d_masked = d.loc[mask].reset_index(drop=True)
        d_test = d_masked.iloc[split:].copy()
        d_test["pred"] = pred

        # 샘플 수가 1 이상인 동만
        grp = d_test.groupby(group_col, observed=True)
        stats = grp.apply(lambda g: pd.Series({
            "n": len(g),
            "mae": mean_absolute_error(g[TARGET].values, g["pred"].values) if len(g) > 0 else np.nan,
            "wmape": (np.sum(np.abs(g[TARGET].values - g["pred"].values)) / (np.sum(np.abs(g[TARGET].values)) + 1e-9)) if len(g) > 0 else np.nan
        }))
        stats = stats.dropna().sort_values("mae")

        if len(stats) == 0:
            print("\n[동별 MAE] 테스트 구간에 유효한 동 데이터가 없습니다. (표본 부족)")
        else:
            print("\n[동별 MAE TOP 10 (좋은 순)]")
            print(stats.head(10))
            print("\n[동별 MAE WORST 10 (나쁜 순)]")
            print(stats.tail(10))

# =====================
# 실행
# =====================
han_res = evaluate_saved_model(HAN_MODEL_PATH, HAN_META_PATH, HAN_DATA_PATH)
del_res = evaluate_saved_model(DEL_MODEL_PATH, DEL_META_PATH, DEL_DATA_PATH)



==== 평가: data/models/han_best_model.pkl ====
Samples: train=354,325 / test=88,582
MAE  : 151,157
RMSE : 1,679,193
R2   : 0.9814
WMAPE: 2.8072%

[동별 MAE TOP 10 (좋은 순)]
                   n           mae     wmape
DONG                                        
수원시 권선구 세류1동  1444.0   5575.550229  0.071540
수원시 권선구 세류3동  2065.0  13811.382568  0.010179
수원시 영통구 매탄4동  2043.0  13914.801772  0.010554
수원시 장안구 정자1동  2028.0  14649.504522  0.008968
수원시 장안구 조원2동  1470.0  15033.683022  0.014717
수원시 팔달구 화서1동  2055.0  17188.697282  0.009409
수원시 권선구 입북동   1865.0  21462.478350  0.015297
수원시 팔달구 매교동   2039.0  21764.044537  0.024374
수원시 팔달구 고등동   2056.0  22144.977556  0.010232
수원시 권선구 권선2동  2062.0  22380.597920  0.011487

[동별 MAE WORST 10 (나쁜 순)]
                   n           mae     wmape
DONG                                        
수원시 영통구 매탄2동  2028.0  9.722877e+04  0.063856
수원시 권선구 권선1동  2070.0  9.830294e+04  0.015604
수원시 영통구 매탄3동  2060.0  1.079290e+05  0.015677
수원시 영통구 광교1동  2059.0  1.771252e+05  0.016

# 3. 매출 데이터 바탕으로 모델 사용

## 점유율

In [29]:
# -*- coding: utf-8 -*-
import re
import joblib
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# =====================================================
# 상수 / 경로
# =====================================================
DATE_COL = "TA_YMD"
TARGET = "AMT"

MY_DATE_COL = "ta_ymd"
MY_AMT_COL = "store_amt"

GRID_XLSX_PATH = "data/수원시 격자.xlsx"

HAN_DATA_PATH = "data/수원시 한식 데이터백업.csv"
DEL_DATA_PATH = "data/수원시 배달 데이터백업.csv"

HAN_MODEL_PATH = "data/models/han_best_model.pkl"
DEL_MODEL_PATH = "data/models/delivery_best_model.pkl"

HAN_META_PATH = "data/models/han_best_meta.pkl"
DEL_META_PATH = "data/models/delivery_best_meta.pkl"


# =====================================================
# 0) 수원시 격자 로드
# =====================================================
def load_suwon_grid_map(xlsx_path=GRID_XLSX_PATH):
    df = pd.read_excel(xlsx_path, header=None)
    df = df.dropna(subset=[0, 1, 2])

    grid = {}
    for _, r in df.iterrows():
        try:
            grid[str(r[0]).strip()] = (int(r[1]), int(r[2]))
        except:
            continue

    if not grid:
        raise ValueError("격자 엑셀에서 nx, ny를 읽지 못했습니다.")
    return grid


def resolve_nxny_from_input_dong(dong, grid_map):
    if dong not in grid_map:
        raise ValueError(f"'{dong}'이(가) 격자 엑셀에 없습니다.")
    return grid_map[dong][0], grid_map[dong][1]


# =====================================================
# 1) 기상청 단기예보 + fallback (과거 평균 날씨)
# =====================================================
def _parse_pcp(v):
    if v in [None, "강수없음", "-", "없음"]:
        return 0.0
    s = str(v).replace("mm", "").strip()
    try:
        return float(s)
    except:
        nums = re.findall(r"[\d.]+", s)
        return float(nums[0]) if nums else 0.0


def fetch_daily_weather_kma(
    nx, ny, target_date, RAIN_ID, base_df, dong
):
    """
    1) 기상청 단기예보 가능 → 사용
    2) NO_DATA / 과거·미래 → 과거 평균 날씨 대체
    """

    d = datetime.strptime(target_date, "%Y-%m-%d").date()
    base_date = (d - timedelta(days=1)).strftime("%Y%m%d")

    url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
    params = {
        "serviceKey": RAIN_ID,
        "dataType": "JSON",
        "numOfRows": 3000,
        "pageNo": 1,
        "base_date": base_date,
        "base_time": "2300",
        "nx": nx,
        "ny": ny,
    }

    try:
        r = requests.get(url, params=params, timeout=10)
        js = r.json()
        body = js.get("response", {}).get("body")

        if body:
            items = body.get("items", {}).get("item", [])
            want = d.strftime("%Y%m%d")

            temps, rains = [], []
            for it in items:
                if it.get("fcstDate") != want:
                    continue
                if it.get("category") == "TMP":
                    temps.append(float(it.get("fcstValue")))
                elif it.get("category") == "PCP":
                    rains.append(_parse_pcp(it.get("fcstValue")))

            if temps:
                return {
                    "temp_mean": float(np.mean(temps)),
                    "rain_mean": float(np.mean(rains)) if rains else 0.0,
                    "rain_peak": float(np.max(rains)) if rains else 0.0,
                    "source": "기상청 단기예보",
                }
    except:
        pass

    # =============================
    # fallback: 과거 평균 날씨
    # =============================
    hist = base_df.copy()
    hist[DATE_COL] = pd.to_datetime(hist[DATE_COL])
    hist = hist[hist["DONG"] == dong]

    temp_cols = [c for c in hist.columns if "temp" in c.lower()]
    rain_cols = [c for c in hist.columns if "rain" in c.lower()]

    return {
        "temp_mean": float(hist[temp_cols[0]].mean()) if temp_cols else np.nan,
        "rain_mean": float(hist[rain_cols[0]].mean()) if rain_cols else 0.0,
        "rain_peak": float(hist[rain_cols[0]].max()) if rain_cols else 0.0,
        "source": "과거 평균 날씨(대체)",
    }


# =====================================================
# 2) 공휴일 (천문연구원)
# =====================================================
def fetch_holiday_flag_kasi(target_date, HOLIDAY_ID):
    d = datetime.strptime(target_date, "%Y-%m-%d")
    url = "https://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo"
    params = {
        "serviceKey": HOLIDAY_ID,
        "solYear": d.year,
        "solMonth": f"{d.month:02d}",
        "_type": "json",
    }

    try:
        r = requests.get(url, params=params, timeout=10)
        items = (
            r.json()
            .get("response", {})
            .get("body", {})
            .get("items", {})
            .get("item", [])
        )

        if isinstance(items, dict):
            items = [items]

        for it in items:
            if str(it.get("locdate")) == d.strftime("%Y%m%d"):
                return {"is_holiday": 1}
    except:
        pass

    return {"is_holiday": 0}


# =====================================================
# 3) 예측용 feature 생성 (❗ DONG 제거됨)
# =====================================================
def build_feature_row_for_prediction(
    base_df, feature_cols, dong, target_date, weather, is_holiday
):
    d = base_df.copy()
    d[DATE_COL] = pd.to_datetime(d[DATE_COL])
    d = d[d["DONG"] == dong].sort_values(DATE_COL)

    td = pd.to_datetime(target_date)
    hist = d[d[DATE_COL] < td]

    row = {
        "year": td.year,
        "month": td.month,
        "day": td.day,
        "dow": td.weekday(),
        "weekend": int(td.weekday() >= 5),
    }

    for lag in [1, 7, 14]:
        row[f"lag_{lag}"] = hist[TARGET].iloc[-lag] if len(hist) >= lag else np.nan

    for win in [7, 14]:
        row[f"roll_mean_{win}"] = hist[TARGET].tail(win).mean() if len(hist) >= win else np.nan
        row[f"roll_std_{win}"] = hist[TARGET].tail(win).std() if len(hist) >= win else np.nan

    row["temp_mean"] = weather["temp_mean"]
    row["rain_mean"] = weather["rain_mean"]
    row["rain_peak"] = weather["rain_peak"]
    row["is_holiday"] = is_holiday

    return pd.DataFrame([{c: row.get(c, np.nan) for c in feature_cols}])


# =====================================================
# 4) 점유율 + 보정계수
# =====================================================
def compute_store_share_and_adj(
    my_store_df, base_df, dong, target_date, lookback_days=60
):
    d0 = datetime.strptime(target_date, "%Y-%m-%d").date()
    start = d0 - timedelta(days=lookback_days)

    ms = my_store_df.copy()
    ms[MY_DATE_COL] = pd.to_datetime(ms[MY_DATE_COL])
    store_amt_60 = ms[
        (ms[MY_DATE_COL].dt.date >= start) &
        (ms[MY_DATE_COL].dt.date < d0)
    ][MY_AMT_COL].sum()

    bd = base_df.copy()
    bd[DATE_COL] = pd.to_datetime(bd[DATE_COL])
    dong_amt_60 = bd[
        (bd["DONG"] == dong) &
        (bd[DATE_COL].dt.date >= start) &
        (bd[DATE_COL].dt.date < d0)
    ][TARGET].sum()

    share = (store_amt_60 / dong_amt_60 * 100) if dong_amt_60 > 0 else 0.0
    return share, 1.0


# =====================================================
# 5) 거래건수 추정
# =====================================================
def estimate_txn_count(my_store_df, target_date, pred_store_amt):
    pseudo_ticket = max(15000.0, pred_store_amt / 20.0)
    return int(pred_store_amt / pseudo_ticket)


# =====================================================
# 6) 메인 함수
# =====================================================
def predict_my_store_sales(
    my_store_csv,
    dong,
    date_str,
    service_type,
    RAIN_ID,
    HOLIDAY_ID,
    grid_xlsx_path=GRID_XLSX_PATH,
):
    if service_type == "delivery":
        model_path, meta_path, data_path = (
            DEL_MODEL_PATH, DEL_META_PATH, DEL_DATA_PATH
        )
    else:
        model_path, meta_path, data_path = (
            HAN_MODEL_PATH, HAN_META_PATH, HAN_DATA_PATH
        )

    model = joblib.load(model_path)
    meta = joblib.load(meta_path)

    # ❗ DONG 제거
    feature_cols = [c for c in meta["feature_cols"] if c != "DONG"]

    base_df = pd.read_csv(data_path)
    my_store_df = pd.read_csv(my_store_csv)

    grid_map = load_suwon_grid_map(grid_xlsx_path)
    nx, ny = resolve_nxny_from_input_dong(dong, grid_map)

    weather = fetch_daily_weather_kma(
        nx, ny, date_str, RAIN_ID, base_df, dong
    )
    holi = fetch_holiday_flag_kasi(date_str, HOLIDAY_ID)

    X_pred = build_feature_row_for_prediction(
        base_df, feature_cols, dong, date_str, weather, holi["is_holiday"]
    )

    pred_dong_amt = float(model.predict(X_pred)[0])
    share, adj = compute_store_share_and_adj(
        my_store_df, base_df, dong, date_str
    )

    pred_store_amt = pred_dong_amt * (share / 100.0) * adj
    pred_cnt = estimate_txn_count(my_store_df, date_str, pred_store_amt)

    # =========================
    # 출력
    # =========================
    print(f"📍지역(동): {dong}")
    print(f"📅날짜: {date_str}")
    print(f"🌦️날씨(출처): {weather['source']}")
    print(f"   - 평균기온: {weather['temp_mean']:.1f}°C")
    print(f"   - 강수(평균/피크): {weather['rain_mean']:.2f} / {weather['rain_peak']:.2f} mm")
    print(f"👥예상 매출건수: {pred_cnt}건")
    print(f"💰예상 동 전체 일매출: {pred_dong_amt:,.0f} 원")
    print(f"🏪가게 점유율(최근 60일): {share:.2f}%")
    print(f"🏪예상 가게 일매출: {pred_store_amt:,.0f} 원")


## 사용예시

In [34]:
import os
from dotenv import load_dotenv
load_dotenv()

holiday_key_env = os.getenv("HOLIDAY_ID")
rain_key_env = os.getenv("RAIN_ID")

predict_my_store_sales(
    my_store_csv="data/my_store.csv",
    dong="행궁동",
    date_str="2025-12-30",
    service_type="han",
    RAIN_ID=rain_key_env,
    HOLIDAY_ID=holiday_key_env,
    grid_xlsx_path="data/수원시 격자.xlsx"
)


📍지역(동): 행궁동
📅날짜: 2025-12-30
🌦️날씨(출처): 과거 평균 날씨(대체)
   - 평균기온: 13.4°C
   - 강수(평균/피크): 0.16 / 37.30 mm
👥예상 매출건수: 0건
💰예상 동 전체 일매출: 84,208 원
🏪가게 점유율(최근 60일): 0.00%
🏪예상 가게 일매출: 0 원


# 4. 모델 재학습

In [27]:
# -*- coding: utf-8 -*-
import os
import joblib
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from datetime import timedelta

# =====================================================
# 설정
# =====================================================
DATE_COL = "TA_YMD"
TARGET = "AMT"

HAN_DATA_PATH = "data/수원시 한식 데이터백업.csv"
DEL_DATA_PATH = "data/수원시 배달 데이터백업.csv"

OUT_DIR = "data/models"
os.makedirs(OUT_DIR, exist_ok=True)

# =====================================================
# 공통 전처리
# =====================================================
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df[DATE_COL] = pd.to_datetime(df[DATE_COL])
    df = df.sort_values(DATE_COL).reset_index(drop=True)

    # ---------- 날짜 파생 ----------
    df["year"] = df[DATE_COL].dt.year
    df["month"] = df[DATE_COL].dt.month
    df["day"] = df[DATE_COL].dt.day
    df["dow"] = df[DATE_COL].dt.weekday
    df["weekend"] = (df["dow"] >= 5).astype(int)

    # ---------- DONG 제거 ----------
    if "DONG" in df.columns:
        df = df.drop(columns=["DONG"])

    # ---------- lag / rolling ----------
    for lag in [1, 7, 14]:
        df[f"lag_{lag}"] = df[TARGET].shift(lag)

    for win in [7, 14]:
        df[f"roll_mean_{win}"] = df[TARGET].rolling(win).mean()
        df[f"roll_std_{win}"] = df[TARGET].rolling(win).std()

    df = df.dropna().reset_index(drop=True)
    return df


# =====================================================
# 학습 함수
# =====================================================
def train_and_save(
    data_path: str,
    model_name: str,
):
    print(f"\n===== TRAIN START: {model_name} =====")

    df = pd.read_csv(data_path)
    df = preprocess(df)

    feature_cols = [c for c in df.columns if c not in [DATE_COL, TARGET]]

    # ---------- time split ----------
    split_date = df[DATE_COL].quantile(0.8)
    train_df = df[df[DATE_COL] <= split_date]
    valid_df = df[df[DATE_COL] > split_date]

    X_train = train_df[feature_cols]
    y_train = train_df[TARGET]
    X_val = valid_df[feature_cols]
    y_val = valid_df[TARGET]

    model = lgb.LGBMRegressor(
        objective="regression_l1",
        n_estimators=5000,
        learning_rate=0.02,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_samples=30,
        random_state=42,
        n_jobs=-1,
    )

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="l1",
        callbacks=[lgb.early_stopping(300, verbose=True)],
    )

    pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, pred)

    print(f"✅ VALID MAE = {mae:,.2f}")

    # ---------- 저장 ----------
    model_path = f"{OUT_DIR}/{model_name}_best_model.pkl"
    meta_path = f"{OUT_DIR}/{model_name}_best_meta.pkl"

    joblib.dump(model, model_path)
    joblib.dump(
        {
            "feature_cols": feature_cols,
            "valid_mae": mae,
            "n_train": len(train_df),
            "n_valid": len(valid_df),
        },
        meta_path,
    )

    print(f"💾 model saved: {model_path}")
    print(f"💾 meta  saved: {meta_path}")

    return mae


# =====================================================
# 실행
# =====================================================
if __name__ == "__main__":
    han_mae = train_and_save(HAN_DATA_PATH, "han")
    del_mae = train_and_save(DEL_DATA_PATH, "delivery")

    print("\n===== DONE =====")
    print(f"HAN MAE     : {han_mae:,.2f}")
    print(f"DELIVERY MAE: {del_mae:,.2f}")



===== TRAIN START: han =====
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009802 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2760
[LightGBM] [Info] Number of data points in the train set: 355094, number of used features: 14
[LightGBM] [Info] Start training from score 2108030.000000
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[5000]	valid_0's l1: 210080
✅ VALID MAE = 210,080.03
💾 model saved: data/models/han_best_model.pkl
💾 meta  saved: data/models/han_best_meta.pkl

===== TRAIN START: delivery =====
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001237 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2500
[LightGBM] [Info] Number of data points in the train set: 30234, number of used features: 14
[LightGBM] [Info] Start training from score

# 5. 정님 모델 수정

In [16]:
import os
import json
import time
import warnings
import joblib
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.base import clone
from xgboost import XGBRegressor

warnings.filterwarnings("ignore")

# =============================
# XGB 사용 가능 여부 체크
# =============================
try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except ImportError:
    HAS_XGB = False


# =============================
# Config
# =============================
HANSIK_PATH = "data/수원시 한식 데이터백업.csv"
DELIV_PATH  = "data/수원시 배달 데이터백업.csv"

SAVE_DIR = "data/model"           
MAX_HOURS = list(range(1, 11))    # 1~10
TEST_RATIO = 0.2                  # 마지막 20% 테스트
RANDOM_STATE = 42

# 원핫 폭발 방지(가능한 sklearn 버전에서만 적용)
OHE_MIN_FREQUENCY = 20
OHE_MAX_CATEGORIES = 200

# UNIT lag/rolling
LAGS = [1, 7, 14]
ROLL_WINDOWS = [7, 14]

# XGB 파라미터(너무 느리면 max_depth/early stopping 줄이세요)
XGB_PARAMS = dict(
    n_estimators=5000,          # 크게 주고 early stopping으로 컷
    learning_rate=0.03,
    max_depth=8,
    min_child_weight=2,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    reg_alpha=0.0,
    objective="reg:absoluteerror",  # MAE 직접 최적화(지원 버전 필요)
    tree_method="hist",
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

ET_PARAMS = dict(
    n_estimators=800,
    max_depth=None,
    min_samples_leaf=2,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

# 누수 위험(예측 시점에 모르는 값) 기본 제외
DROP_COLS = ["UNIT", "DATE", "AMT", "CNT"]


# =============================
# Utils
# =============================
def evaluate_reg(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    return float(mae), float(rmse), float(r2)


def safe_onehot_encoder():
    """
    sklearn 버전에 따라 min_frequency / max_categories 미지원일 수 있어 예외 처리.
    """
    try:
        return OneHotEncoder(
            handle_unknown="ignore",
            min_frequency=OHE_MIN_FREQUENCY,
            max_categories=OHE_MAX_CATEGORIES
        )
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore")


def build_preprocess(X: pd.DataFrame):
    """
    X의 dtype 기반으로 ColumnTransformer 생성
    """
    cat_cols = [c for c in X.columns if X[c].dtype == "object"]
    num_cols = [c for c in X.columns if c not in cat_cols]

    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ])

    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", safe_onehot_encoder())
    ])

    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_cols),
            ("cat", categorical_transformer, cat_cols)
        ],
        remainder="drop"
    )
    return preprocess


# =============================
# 1) Load
# =============================
hansik = pd.read_csv(HANSIK_PATH)
deliv  = pd.read_csv(DELIV_PATH)

required = {"TA_YMD", "DONG", "HOUR", "DAY", "UNIT"}
missing = required - set(hansik.columns)
if missing:
    raise ValueError(f"[한식 데이터] 필수 컬럼이 없습니다: {missing}")

missing2 = {"TA_YMD", "DONG", "HOUR", "DAY"} - set(deliv.columns)
if missing2:
    raise ValueError(f"[배달 데이터] 필수 컬럼이 없습니다: {missing2}")


# =============================
# 2) Merge DELIV as features
# =============================
merge_keys = ["TA_YMD", "DONG", "HOUR", "DAY"]

deliv_feat = deliv.copy()
for c in deliv_feat.columns:
    if c not in merge_keys:
        deliv_feat.rename(columns={c: f"DELIV_{c}"}, inplace=True)

df = hansik.merge(deliv_feat, on=merge_keys, how="left")


# =============================
# 3) Date features
# =============================
df["TA_YMD"] = df["TA_YMD"].astype(str)
df["DATE"] = pd.to_datetime(df["TA_YMD"], format="%Y%m%d", errors="coerce")
df = df[df["DATE"].notna()].copy()

df["YEAR"] = df["DATE"].dt.year
df["MONTH"] = df["DATE"].dt.month
df["DAY_OF_MONTH"] = df["DATE"].dt.day
df["WEEKOFYEAR"] = df["DATE"].dt.isocalendar().week.astype(int)


# =============================
# 4) Lag/Rolling features for UNIT (no leakage if time-split)
#    group key: (DONG, HOUR)
# =============================
df = df.sort_values(["DONG", "HOUR", "DATE"]).reset_index(drop=True)
grp = df.groupby(["DONG", "HOUR"], sort=False)

for lag in LAGS:
    df[f"UNIT_LAG_{lag}"] = grp["UNIT"].shift(lag)

for w in ROLL_WINDOWS:
    df[f"UNIT_ROLL_MEAN_{w}"] = grp["UNIT"].shift(1).rolling(window=w, min_periods=max(2, w // 3)).mean()
    df[f"UNIT_ROLL_STD_{w}"]  = grp["UNIT"].shift(1).rolling(window=w, min_periods=max(2, w // 3)).std()

df["UNIT_LAG_1_MISSING"] = df["UNIT_LAG_1"].isna().astype(int)

# 타겟
y_all = df["UNIT"].copy()

# 피처
X_all = df.drop(columns=[c for c in DROP_COLS if c in df.columns], errors="ignore")


# =============================
# 5) Train per HOUR
# =============================
os.makedirs(SAVE_DIR, exist_ok=True)

hours = sorted(pd.unique(df["HOUR"].dropna()))
hours = [int(h) for h in hours if int(h) in MAX_HOURS]

print(f"[INFO] XGB 사용 여부: {HAS_XGB}")
print(f"[INFO] 학습 시간대: {hours}")
print(f"[INFO] 저장 폴더: {SAVE_DIR}")



all_results = []
t0 = time.time()

print("\n[DEBUG] >>> 학습 루프 시작 직전")
for h in hours:
    idx = (df["HOUR"].astype(int) == h)
    X_h = X_all.loc[idx].copy()
    y_h = y_all.loc[idx].copy()
    d_h = df.loc[idx, "DATE"].copy()

    if len(X_h) < 800:
        print(f"\n[SKIP] HOUR={h:02d}: 데이터가 너무 적음 (n={len(X_h)})")
        continue

    # 날짜 정렬
    order = np.argsort(d_h.values)
    X_h = X_h.iloc[order].reset_index(drop=True)
    y_h = y_h.iloc[order].reset_index(drop=True)

    n = len(X_h)
    split = int(n * (1 - TEST_RATIO))
    X_train, X_test = X_h.iloc[:split], X_h.iloc[split:]
    y_train, y_test = y_h.iloc[:split], y_h.iloc[split:]

    # XGB early stopping용 valid (train의 마지막 10%)
    split2 = int(len(X_train) * 0.9)
    X_tr, X_val = X_train.iloc[:split2], X_train.iloc[split2:]
    y_tr, y_val = y_train.iloc[:split2], y_train.iloc[split2:]

    print(f"\n===== HOUR={h:02d} | train={len(X_train)} test={len(X_test)} =====")

    hour_dir = os.path.join(SAVE_DIR, f"HOUR_{h:02d}")
    os.makedirs(hour_dir, exist_ok=True)

    candidates = []

    # -------------------------
    # A방식: preprocess를 먼저 fit/transform 후 모델 학습
    # -------------------------
    preprocess = build_preprocess(X_train)
    pre = clone(preprocess)

    X_tr_enc  = pre.fit_transform(X_tr)
    X_val_enc = pre.transform(X_val)
    X_test_enc = pre.transform(X_test)

    # --- 1) XGB (있으면 가장 강력) ---
    if HAS_XGB:
        # objective가 버전에 따라 에러날 수 있어 fallback 처리
        xgb_params = dict(XGB_PARAMS)
        try:
            model_xgb = XGBRegressor(**xgb_params)
            model_xgb.fit(
                X_tr_enc, y_tr,
                eval_set=[(X_val_enc, y_val)],
                verbose=False,
                early_stopping_rounds=200
            )
        except Exception as e:
            # fallback: objective를 reg:squarederror로
            xgb_params["objective"] = "reg:squarederror"
            model_xgb = XGBRegressor(**xgb_params)
            model_xgb.fit(
                X_tr_enc, y_tr,
                eval_set=[(X_val_enc, y_val)],
                verbose=False,
                early_stopping_rounds=200
            )

        pred = model_xgb.predict(X_test_enc)
        mae, rmse, r2 = evaluate_reg(y_test, pred)
        print(f"- XGB       | MAE={mae:,.3f} RMSE={rmse:,.3f} R2={r2:,.4f}")

        bundle = {"preprocess": pre, "model": model_xgb, "model_name": "XGB"}
        candidates.append(("XGB", bundle, mae, rmse, r2))

    # --- 2) ExtraTrees (빠른/안정 대안) ---
    model_et = ExtraTreesRegressor(**ET_PARAMS)
    model_et.fit(X_tr_enc, y_tr)  # valid는 사용 안 함(빠름)

    pred = model_et.predict(X_test_enc)
    mae, rmse, r2 = evaluate_reg(y_test, pred)
    print(f"- EXTRATREE | MAE={mae:,.3f} RMSE={rmse:,.3f} R2={r2:,.4f}")

    bundle = {"preprocess": pre, "model": model_et, "model_name": "EXTRATREE"}
    candidates.append(("EXTRATREE", bundle, mae, rmse, r2))

    # --- pick best by MAE ---
    candidates.sort(key=lambda x: x[2])
    best_name, best_bundle, best_mae, best_rmse, best_r2 = candidates[0]

    best_path = os.path.join(hour_dir, "best_model.joblib")
    joblib.dump(best_bundle, best_path)

    meta = {
        "HOUR": int(h),
        "BEST_MODEL": best_name,
        "MAE": float(best_mae),
        "RMSE": float(best_rmse),
        "R2": float(best_r2),
        "TRAIN_SIZE": int(len(X_train)),
        "TEST_SIZE": int(len(X_test)),
        "FEATURE_COLS": list(X_all.columns),
        "DROP_COLS": DROP_COLS,
        "LAGS": LAGS,
        "ROLL_WINDOWS": ROLL_WINDOWS,
        "HAS_XGB": bool(HAS_XGB),
        "XGB_PARAMS": XGB_PARAMS if HAS_XGB else None,
        "ET_PARAMS": ET_PARAMS,
    }
    with open(os.path.join(hour_dir, "meta.json"), "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    print(f"✅ BEST => {best_name} | MAE={best_mae:,.3f} 저장: {best_path}")

    all_results.append({
        "HOUR": int(h),
        "BEST_MODEL": best_name,
        "MAE": float(best_mae),
        "RMSE": float(best_rmse),
        "R2": float(best_r2),
        "SAVE_PATH": best_path
    })

# 결과 저장
res_df = pd.DataFrame(all_results).sort_values(["HOUR"])
res_path = os.path.join(SAVE_DIR, "results_by_hour.csv")
res_df.to_csv(res_path, index=False, encoding="utf-8-sig")

print("\n[DONE] 학습 완료")
print(f"- 결과표 저장: {res_path}")
print(f"- 모델 저장 폴더: {SAVE_DIR}")
print(f"- 총 소요(초): {time.time() - t0:,.1f}")


# =============================
# 6) 로드/예측 헬퍼
# =============================
def load_best_bundle(hour: int):
    hour_dir = os.path.join(SAVE_DIR, f"HOUR_{hour:02d}")
    path = os.path.join(hour_dir, "best_model.joblib")
    return joblib.load(path)

def predict_with_bundle(bundle: dict, X_new: pd.DataFrame) -> np.ndarray:
    pre = bundle["preprocess"]
    model = bundle["model"]
    X_enc = pre.transform(X_new)
    return model.predict(X_enc)

"""
사용 예:
bundle = load_best_bundle(5)
sample = X_all.iloc[[0]].copy()          # 컬럼 구조 동일해야 함
yhat = predict_with_bundle(bundle, sample)
print(yhat)
"""


[INFO] XGB 사용 여부: True
[INFO] 학습 시간대: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[INFO] 저장 폴더: data/model

[DEBUG] >>> 학습 루프 시작 직전

[DEBUG] >>> HOUR=01 루프 진입
[DEBUG] HOUR=01 rows=42419
[DEBUG] HOUR=01 enc shapes: tr=(30541, 267), val=(3394, 267), test=(8484, 267)
[DEBUG] HOUR=01 DONE | MAE=16813.879
[DEBUG] ✅ SAVED => data/model\HOUR_01\best_model.joblib (exists=True)

[DEBUG] >>> HOUR=02 루프 진입
[DEBUG] HOUR=02 rows=42498
[DEBUG] HOUR=02 enc shapes: tr=(30598, 267), val=(3400, 267), test=(8500, 267)
[DEBUG] HOUR=02 DONE | MAE=8036.923
[DEBUG] ✅ SAVED => data/model\HOUR_02\best_model.joblib (exists=True)

[DEBUG] >>> HOUR=03 루프 진입
[DEBUG] HOUR=03 rows=44644
[DEBUG] HOUR=03 enc shapes: tr=(32143, 267), val=(3572, 267), test=(8929, 267)
[DEBUG] HOUR=03 DONE | MAE=8344.425
[DEBUG] ✅ SAVED => data/model\HOUR_03\best_model.joblib (exists=True)

[DEBUG] >>> HOUR=04 루프 진입
[DEBUG] HOUR=04 rows=45451
[DEBUG] HOUR=04 enc shapes: tr=(32724, 267), val=(3636, 267), test=(9091, 267)
[DEBUG] HOUR=04 DONE | MAE=42

'\n사용 예:\nbundle = load_best_bundle(5)\nsample = X_all.iloc[[0]].copy()          # 컬럼 구조 동일해야 함\nyhat = predict_with_bundle(bundle, sample)\nprint(yhat)\n'

## 사용

In [7]:
import os
import joblib
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta

# =========================
# 설정
# =========================
from dotenv import load_dotenv
load_dotenv()  

SERVICE_KEY = os.getenv("RAIN_ID")  # 기상청 단기예보 서비스키

MODELS_ROOT = "data/model"
BEST_MAP_PATH = os.path.join(MODELS_ROOT, "results_by_hour.csv")  
BUNDLE_PATH_TEMPLATE = os.path.join(MODELS_ROOT, "HOUR_{hour:02d}", "best_model.joblib")  

# 학습 때 로그타겟이면 True
USE_LOG_TARGET = False

# (학습 때와 동일해야 함) 안전장치로 drop
DROP_COLS = ["UNIT", "DATE", "AMT", "CNT"]

# =========================
# 항목요약 시간대(01~10)
# =========================
HOUR_BINS = {
    1: list(range(0, 7)),     
    2: [7, 8],
    3: [9, 10],
    4: [11, 12],
    5: [13, 14],
    6: [15, 16],
    7: [17, 18],
    8: [19, 20],
    9: [21, 22],
    10: [23],
}

# =========================
# 날짜 파생 + DAY(01=월..07=일)
# =========================
def compute_day_code(date_yyyymmdd: str) -> int:
    dt = pd.to_datetime(str(date_yyyymmdd), format="%Y%m%d", errors="raise")
    return int(dt.weekday() + 1)

def make_date_features(date_yyyymmdd: str) -> dict:
    dt = pd.to_datetime(str(date_yyyymmdd), format="%Y%m%d", errors="raise")
    return {
        "TA_YMD": str(date_yyyymmdd),
        "YEAR": int(dt.year),
        "MONTH": int(dt.month),
        "DAY_OF_MONTH": int(dt.day),
        "WEEKOFYEAR": int(dt.isocalendar().week),
    }

# =========================
# 기상청 단기예보 TMP/PCP
# =========================
def _parse_pcp_to_mm(x):
    if x is None:
        return 0.0
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).strip()
    if s in ("강수없음", "없음", "", "nan", "NaN"):
        return 0.0
    if "미만" in s:
        try:
            num = float(s.replace("mm", "").replace("미만", "").strip())
            return max(0.0, num * 0.5)
        except:
            return 0.0
    if "~" in s:
        try:
            a, b = s.replace("mm", "").split("~")
            return (float(a) + float(b)) / 2.0
        except:
            return 0.0
    try:
        return float(s.replace("mm", ""))
    except:
        return 0.0

def fetch_vilage_fcst_json(service_key, base_date, base_time, nx, ny, num_rows=3000):
    url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
    params = {
        "serviceKey": service_key,
        "pageNo": 1,
        "numOfRows": num_rows,
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": nx,
        "ny": ny,
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

def fetch_hourly_tmp_pcp_for_date(service_key, target_yyyymmdd, nx, ny):
    base_times = ["2300","2000","1700","1400","1100","0800","0500","0200"]
    now = datetime.now()
    today = now.strftime("%Y%m%d")
    yesterday = (now - timedelta(days=1)).strftime("%Y%m%d")

    last_err = None
    for bd in [today, yesterday]:
        for bt in base_times:
            try:
                js = fetch_vilage_fcst_json(service_key, bd, bt, nx, ny)
                items = js["response"]["body"]["items"]["item"]
                df = pd.DataFrame(items)

                df = df[df["category"].isin(["TMP", "PCP"])]
                df = df[df["fcstDate"].astype(str) == str(target_yyyymmdd)]
                if df.empty:
                    continue

                df["fcstTime"] = df["fcstTime"].astype(str).str.zfill(4)
                piv = df.pivot_table(
                    index="fcstTime",
                    columns="category",
                    values="fcstValue",
                    aggfunc="first"
                ).reset_index()

                if "TMP" not in piv.columns:
                    continue

                piv["TMP"] = piv["TMP"].astype(float)
                piv["PCP"] = piv["PCP"].apply(_parse_pcp_to_mm) if "PCP" in piv.columns else 0.0
                piv["PCP"] = piv["PCP"].astype(float)
                piv["base_used"] = f"{bd}-{bt}"
                return piv.sort_values("fcstTime").reset_index(drop=True)
            except Exception as e:
                last_err = e
                continue

    raise RuntimeError(f"해당 날짜의 기상청 예보 데이터를 찾지 못했습니다. last_err={last_err}")

def aggregate_weather_to_item_slots(hourly_df: pd.DataFrame) -> dict:
    hourly_df = hourly_df.copy()
    hourly_df["HOUR_OF_DAY"] = hourly_df["fcstTime"].str[:2].astype(int)

    out = {}
    for h, hours in HOUR_BINS.items():
        sub = hourly_df[hourly_df["HOUR_OF_DAY"].isin(hours)]
        out[h] = {
            "TEMP": float(sub["TMP"].mean()) if not sub.empty else np.nan,
            "RAIN": float(sub["PCP"].sum()) if not sub.empty else 0.0,
        }

    temps = [v["TEMP"] for v in out.values() if not pd.isna(v["TEMP"])]
    fill_temp = float(np.mean(temps)) if temps else 0.0
    for h in out:
        if pd.isna(out[h]["TEMP"]):
            out[h]["TEMP"] = fill_temp

    return out

# =========================
# 모델 로드 + "모델이 기대하는 컬럼" 자동 맞춤
# (학습 저장물: best_model.joblib = {"preprocess": ColumnTransformer, "model": estimator, "model_name": str})
# =========================
def load_best_model_map():
    if not os.path.exists(BEST_MAP_PATH):
        raise FileNotFoundError(f"결과 파일이 없습니다: {BEST_MAP_PATH}")
    df = pd.read_csv(BEST_MAP_PATH)
    required_cols = {"HOUR", "BEST_MODEL"}
    if not required_cols.issubset(df.columns):
        raise ValueError(f"{BEST_MAP_PATH}에 필요한 컬럼이 없습니다. 필요: {required_cols}, 현재: {set(df.columns)}")
    return df.set_index("HOUR")["BEST_MODEL"].to_dict()

def load_best_bundle(hour: int):
    path = BUNDLE_PATH_TEMPLATE.format(hour=hour)
    if not os.path.exists(path):
        raise FileNotFoundError(f"번들 파일이 없습니다: {path}")
    bundle = joblib.load(path)
    # 최소 검증
    if not isinstance(bundle, dict) or "preprocess" not in bundle or "model" not in bundle:
        raise ValueError(f"번들 포맷이 예상과 다릅니다: {path}")
    return bundle

def get_required_feature_names_from_preprocess(preprocess) -> list:
    """
    ColumnTransformer가 기대하는 원본 feature명 추출
    (fit 이후 preprocess.transformers_에 cols가 들어있음)
    """
    names = []
    for name, trans, cols in getattr(preprocess, "transformers_", []):
        if cols is None:
            continue
        if isinstance(cols, (list, tuple, np.ndarray, pd.Index)):
            names.extend(list(cols))
    return sorted(set([str(x) for x in names]))

def align_row_to_required_columns(X_row: pd.DataFrame, required_cols: list) -> pd.DataFrame:
    """
    required_cols에 없는 컬럼은 제거, 없는 컬럼은 생성(NaN).
    imputer가 있으니 NaN 생성이 안전.
    """
    X = X_row.copy()

    if required_cols:
        # 제거
        drop_cols = [c for c in X.columns if c not in required_cols]
        if drop_cols:
            X = X.drop(columns=drop_cols, errors="ignore")

        # 생성
        for c in required_cols:
            if c not in X.columns:
                X[c] = np.nan

        # 순서 고정
        X = X[required_cols]

    return X

def predict_with_bundle(bundle: dict, X_new: pd.DataFrame) -> np.ndarray:
    pre = bundle["preprocess"]
    model = bundle["model"]
    X_enc = pre.transform(X_new)
    return model.predict(X_enc)

# =========================
# 핵심: 날짜 1개 → 1~10시간대 예측
# - 기상청 TEMP/RAIN을 항목요약 시간대로 집계해서 넣음
# - 시간대별 "best_model.joblib" 자동 로드
# - preprocess가 기대하는 컬럼 자동 보정
# =========================
def predict_date_all_hours_with_weather(
    date_yyyymmdd: str,
    nx: int,
    ny: int,
    service_key: str = SERVICE_KEY
) -> pd.DataFrame:
    if not service_key:
        raise ValueError("SERVICE_KEY(RAIN_ID)가 비어있습니다. .env 또는 환경변수를 확인하세요.")

    day_code = compute_day_code(date_yyyymmdd)

    hourly_weather = fetch_hourly_tmp_pcp_for_date(service_key, date_yyyymmdd, nx, ny)
    weather_by_hour = aggregate_weather_to_item_slots(hourly_weather)

    best_map = load_best_model_map()
    rows = []

    for hour in range(1, 11):
        if hour not in best_map:
            continue

        # 학습 코드에서는 best_model.joblib 한 개만 저장 (BEST_MODEL 명은 meta용)
        bundle = load_best_bundle(hour)
        best_name = str(best_map[hour])

        # 기본 row: 날짜/시간대/요일 + 날씨
        row = {}
        row.update(make_date_features(date_yyyymmdd))
        row.update({
            "HOUR": int(hour),
            "DAY": int(day_code),

            # (학습 데이터에 존재한다면 자동 사용될 수 있게)
            "TEMP": float(weather_by_hour[hour]["TEMP"]),
            "RAIN": float(weather_by_hour[hour]["RAIN"]),

            # merge된 배달 피처명을 대비해서도 넣어둠(학습에서 DELIV_*가 있었음)
            "DELIV_TEMP": float(weather_by_hour[hour]["TEMP"]),
            "DELIV_RAIN": float(weather_by_hour[hour]["RAIN"]),
        })

        X_row = pd.DataFrame([row])
        X_row = X_row.drop(columns=[c for c in DROP_COLS if c in X_row.columns], errors="ignore")

        # preprocess가 기대하는 원본 컬럼으로 맞춤
        required = get_required_feature_names_from_preprocess(bundle["preprocess"])
        X_row_aligned = align_row_to_required_columns(X_row, required)

        pred = float(predict_with_bundle(bundle, X_row_aligned)[0])
        if USE_LOG_TARGET:
            pred = float(np.expm1(pred))

        rows.append({
            "TA_YMD": str(date_yyyymmdd),
            "DAY": int(day_code),
            "HOUR": int(hour),
            "BEST_MODEL": best_name,             
            "TEMP": round(float(row["TEMP"]), 2),
            "RAIN": float(row["RAIN"]),
            "PRED_UNIT": int(round(pred)),
            "KMA_BASE_USED": str(hourly_weather.loc[0, "base_used"]),
        })

    return pd.DataFrame(rows).sort_values("HOUR").reset_index(drop=True)

# =========================
# 사용 예시
# =========================
if __name__ == "__main__":
    df_pred = predict_date_all_hours_with_weather(
        date_yyyymmdd="20251230",
        nx=61,
        ny=121
    )
    print(df_pred)


     TA_YMD  DAY  HOUR BEST_MODEL  TEMP  RAIN  PRED_UNIT  KMA_BASE_USED
0  20251230    2     1        XGB -0.86   0.0      38545  20251230-1100
1  20251230    2     2        XGB -0.86   0.0      11827  20251230-1100
2  20251230    2     3        XGB -0.86   0.0      16934  20251230-1100
3  20251230    2     4  EXTRATREE  1.00   0.0      25555  20251230-1100
4  20251230    2     5  EXTRATREE  1.50   0.0      24160  20251230-1100
5  20251230    2     6        XGB  1.00   0.0      23881  20251230-1100
6  20251230    2     7  EXTRATREE -0.50   0.0      33109  20251230-1100
7  20251230    2     8  EXTRATREE -2.00   0.0      38352  20251230-1100
8  20251230    2     9        XGB -3.00   0.0      51828  20251230-1100
9  20251230    2    10        XGB -4.00   0.0      51965  20251230-1100


## 동입력후 사용

In [12]:
import os
import joblib
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta

# =========================
# 설정
# =========================
from dotenv import load_dotenv
load_dotenv()

SERVICE_KEY = os.getenv("RAIN_ID")  # 기상청 단기예보 서비스키

MODELS_ROOT = "data/model"
BEST_MAP_PATH = os.path.join(MODELS_ROOT, "results_by_hour.csv")
BUNDLE_PATH_TEMPLATE = os.path.join(MODELS_ROOT, "HOUR_{hour:02d}", "best_model.joblib")

# 학습 때 로그타겟이면 True
USE_LOG_TARGET = False

# (학습 때와 동일해야 함) 안전장치로 drop
DROP_COLS = ["UNIT", "DATE", "AMT", "CNT"]

# 격자 엑셀
GRID_XLSX_PATH = "data/수원시 격자.xlsx"
_GRID_CACHE = None  # 반복 로딩 방지 캐시

# =========================
# 항목요약 시간대(01~10)
# =========================
HOUR_BINS = {
    1: list(range(0, 7)),     # 00~06
    2: [7, 8],
    3: [9, 10],
    4: [11, 12],
    5: [13, 14],
    6: [15, 16],
    7: [17, 18],
    8: [19, 20],
    9: [21, 22],
    10: [23],
}

# =========================
# 날짜 파생 + DAY(01=월..07=일)
# =========================
def compute_day_code(date_yyyymmdd: str) -> int:
    dt = pd.to_datetime(str(date_yyyymmdd), format="%Y%m%d", errors="raise")
    return int(dt.weekday() + 1)

def make_date_features(date_yyyymmdd: str) -> dict:
    dt = pd.to_datetime(str(date_yyyymmdd), format="%Y%m%d", errors="raise")
    return {
        "TA_YMD": str(date_yyyymmdd),
        "YEAR": int(dt.year),
        "MONTH": int(dt.month),
        "DAY_OF_MONTH": int(dt.day),
        "WEEKOFYEAR": int(dt.isocalendar().week),
    }

# =========================
# 격자: DONG -> (nx, ny)
# =========================
def load_grid_table(path: str = GRID_XLSX_PATH) -> pd.DataFrame:
    """
    data/수원시 격자.xlsx
    - A열: 동이름, B열: nx, C열: ny (사용자 설명 기준)
    - header가 없다고 가정하고 A/B/C를 고정으로 읽음
      (만약 실제 파일에 헤더가 있으면 header=None -> header=0 으로 변경)
    """
    global _GRID_CACHE
    if _GRID_CACHE is not None:
        return _GRID_CACHE

    if not os.path.exists(path):
        raise FileNotFoundError(f"격자 파일이 없습니다: {path}")

    df = pd.read_excel(path, header=None)
    if df.shape[1] < 3:
        raise ValueError(f"격자 파일 컬럼 수가 부족합니다(A,B,C 필요): {path}")

    df = df.iloc[:, :3].copy()
    df.columns = ["DONG_NAME", "nx", "ny"]

    df["DONG_NAME"] = df["DONG_NAME"].astype(str).str.strip()
    df["nx"] = pd.to_numeric(df["nx"], errors="coerce")
    df["ny"] = pd.to_numeric(df["ny"], errors="coerce")
    df = df.dropna(subset=["nx", "ny"]).copy()
    df["nx"] = df["nx"].astype(int)
    df["ny"] = df["ny"].astype(int)

    _GRID_CACHE = df
    return df

def find_grid_by_dong(dong: str, path: str = GRID_XLSX_PATH) -> tuple[int, int]:
    """
    사용자가 입력한 dong(예: '행궁동', '수원시 팔달구 행궁동')으로
    엑셀 A열(DONG_NAME)에서 격자(nx,ny) 찾기.
    - 완전일치 우선, 없으면 포함검색(양방향)
    - 후보가 여러 개면 DONG_NAME 길이가 짧은 것을 우선 선택
    """
    dong = str(dong).strip()
    if not dong:
        raise ValueError("dong(동) 입력이 비었습니다.")

    grid = load_grid_table(path)

    # 1) 완전일치
    exact = grid[grid["DONG_NAME"] == dong]
    if not exact.empty:
        r = exact.iloc[0]
        return int(r["nx"]), int(r["ny"])

    # 2) 포함검색(양방향)
    a = grid["DONG_NAME"].str.contains(dong, na=False)
    b = grid["DONG_NAME"].apply(lambda x: str(dong).find(str(x)) >= 0)
    cand = grid[a | b].copy()

    # fallback: 마지막 토큰(보통 'OO동')로 재시도
    if cand.empty:
        tok = dong.split()[-1]
        cand = grid[grid["DONG_NAME"].str.contains(tok, na=False)].copy()

    if cand.empty:
        raise ValueError(f"'{dong}'에 해당하는 격자(nx,ny)를 찾지 못했습니다. 엑셀 A열 동이름을 확인하세요.")

    cand["name_len"] = cand["DONG_NAME"].str.len()
    cand = cand.sort_values(["name_len"]).reset_index(drop=True)
    r = cand.iloc[0]
    return int(r["nx"]), int(r["ny"])

# =========================
# 기상청 단기예보 TMP/PCP
# =========================
def _parse_pcp_to_mm(x):
    if x is None:
        return 0.0
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).strip()
    if s in ("강수없음", "없음", "", "nan", "NaN"):
        return 0.0
    if "미만" in s:
        try:
            num = float(s.replace("mm", "").replace("미만", "").strip())
            return max(0.0, num * 0.5)
        except:
            return 0.0
    if "~" in s:
        try:
            a, b = s.replace("mm", "").split("~")
            return (float(a) + float(b)) / 2.0
        except:
            return 0.0
    try:
        return float(s.replace("mm", ""))
    except:
        return 0.0

def fetch_vilage_fcst_json(service_key, base_date, base_time, nx, ny, num_rows=3000):
    url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
    params = {
        "serviceKey": service_key,
        "pageNo": 1,
        "numOfRows": num_rows,
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": nx,
        "ny": ny,
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

def fetch_hourly_tmp_pcp_for_date(service_key, target_yyyymmdd, nx, ny):
    base_times = ["2300", "2000", "1700", "1400", "1100", "0800", "0500", "0200"]
    now = datetime.now()
    today = now.strftime("%Y%m%d")
    yesterday = (now - timedelta(days=1)).strftime("%Y%m%d")

    last_err = None
    for bd in [today, yesterday]:
        for bt in base_times:
            try:
                js = fetch_vilage_fcst_json(service_key, bd, bt, nx, ny)
                items = js["response"]["body"]["items"]["item"]
                df = pd.DataFrame(items)

                df = df[df["category"].isin(["TMP", "PCP"])]
                df = df[df["fcstDate"].astype(str) == str(target_yyyymmdd)]
                if df.empty:
                    continue

                df["fcstTime"] = df["fcstTime"].astype(str).str.zfill(4)
                piv = df.pivot_table(
                    index="fcstTime",
                    columns="category",
                    values="fcstValue",
                    aggfunc="first"
                ).reset_index()

                if "TMP" not in piv.columns:
                    continue

                piv["TMP"] = piv["TMP"].astype(float)
                piv["PCP"] = piv["PCP"].apply(_parse_pcp_to_mm) if "PCP" in piv.columns else 0.0
                piv["PCP"] = piv["PCP"].astype(float)
                piv["base_used"] = f"{bd}-{bt}"
                return piv.sort_values("fcstTime").reset_index(drop=True)
            except Exception as e:
                last_err = e
                continue

    raise RuntimeError(f"해당 날짜의 기상청 예보 데이터를 찾지 못했습니다. last_err={last_err}")

def aggregate_weather_to_item_slots(hourly_df: pd.DataFrame) -> dict:
    hourly_df = hourly_df.copy()
    hourly_df["HOUR_OF_DAY"] = hourly_df["fcstTime"].str[:2].astype(int)

    out = {}
    for h, hours in HOUR_BINS.items():
        sub = hourly_df[hourly_df["HOUR_OF_DAY"].isin(hours)]
        out[h] = {
            "TEMP": float(sub["TMP"].mean()) if not sub.empty else np.nan,
            "RAIN": float(sub["PCP"].sum()) if not sub.empty else 0.0,
        }

    temps = [v["TEMP"] for v in out.values() if not pd.isna(v["TEMP"])]
    fill_temp = float(np.mean(temps)) if temps else 0.0
    for h in out:
        if pd.isna(out[h]["TEMP"]):
            out[h]["TEMP"] = fill_temp

    return out

# =========================
# ✅ 동 입력 -> 그 동의 "기온" 반환(원하는 함수)
# =========================
def get_temperature_by_dong(
    date_yyyymmdd: str,
    dong: str,
    slot_hour: int = 5,
    service_key: str = SERVICE_KEY
) -> float:
    """
    dong 입력 -> 격자(nx,ny) 조회 -> 기상청 예보 -> slot_hour(1~10)의 TEMP 반환
    """
    if not service_key:
        raise ValueError("SERVICE_KEY(RAIN_ID)가 비어있습니다. .env 또는 환경변수를 확인하세요.")
    if not (1 <= int(slot_hour) <= 10):
        raise ValueError("slot_hour는 1~10 이어야 합니다.")

    nx, ny = find_grid_by_dong(dong)
    hourly_weather = fetch_hourly_tmp_pcp_for_date(service_key, date_yyyymmdd, nx, ny)
    weather_by_hour = aggregate_weather_to_item_slots(hourly_weather)

    temp = float(weather_by_hour[int(slot_hour)]["TEMP"])
    return round(temp, 1)  # ✅ 소수점 1자리 반환

# =========================
# 모델 로드 + "모델이 기대하는 컬럼" 자동 맞춤
# (학습 저장물: best_model.joblib = {"preprocess": ColumnTransformer, "model": estimator, "model_name": str})
# =========================
def load_best_model_map():
    if not os.path.exists(BEST_MAP_PATH):
        raise FileNotFoundError(f"결과 파일이 없습니다: {BEST_MAP_PATH}")
    df = pd.read_csv(BEST_MAP_PATH)
    required_cols = {"HOUR", "BEST_MODEL"}
    if not required_cols.issubset(df.columns):
        raise ValueError(f"{BEST_MAP_PATH}에 필요한 컬럼이 없습니다. 필요: {required_cols}, 현재: {set(df.columns)}")
    return df.set_index("HOUR")["BEST_MODEL"].to_dict()

def load_best_bundle(hour: int):
    path = BUNDLE_PATH_TEMPLATE.format(hour=hour)
    if not os.path.exists(path):
        raise FileNotFoundError(f"번들 파일이 없습니다: {path}")
    bundle = joblib.load(path)
    if not isinstance(bundle, dict) or "preprocess" not in bundle or "model" not in bundle:
        raise ValueError(f"번들 포맷이 예상과 다릅니다: {path}")
    return bundle

def get_required_feature_names_from_preprocess(preprocess) -> list:
    names = []
    for name, trans, cols in getattr(preprocess, "transformers_", []):
        if cols is None:
            continue
        if isinstance(cols, (list, tuple, np.ndarray, pd.Index)):
            names.extend(list(cols))
    return sorted(set([str(x) for x in names]))

def align_row_to_required_columns(X_row: pd.DataFrame, required_cols: list) -> pd.DataFrame:
    X = X_row.copy()

    if required_cols:
        drop_cols = [c for c in X.columns if c not in required_cols]
        if drop_cols:
            X = X.drop(columns=drop_cols, errors="ignore")

        for c in required_cols:
            if c not in X.columns:
                X[c] = np.nan

        X = X[required_cols]

    return X

def predict_with_bundle(bundle: dict, X_new: pd.DataFrame) -> np.ndarray:
    pre = bundle["preprocess"]
    model = bundle["model"]
    X_enc = pre.transform(X_new)
    return model.predict(X_enc)

# =========================
# 핵심: 날짜 1개 → 1~10시간대 예측
# - (nx, ny 직접 입력 버전)
# =========================
def predict_date_all_hours_with_weather(
    date_yyyymmdd: str,
    nx: int,
    ny: int,
    service_key: str = SERVICE_KEY
) -> pd.DataFrame:
    if not service_key:
        raise ValueError("SERVICE_KEY(RAIN_ID)가 비어있습니다. .env 또는 환경변수를 확인하세요.")

    day_code = compute_day_code(date_yyyymmdd)

    hourly_weather = fetch_hourly_tmp_pcp_for_date(service_key, date_yyyymmdd, nx, ny)
    weather_by_hour = aggregate_weather_to_item_slots(hourly_weather)

    best_map = load_best_model_map()
    rows = []

    for hour in range(1, 11):
        if hour not in best_map:
            continue

        bundle = load_best_bundle(hour)
        best_name = str(best_map[hour])

        row = {}
        row.update(make_date_features(date_yyyymmdd))
        row.update({
            "HOUR": int(hour),
            "DAY": int(day_code),
            "TEMP": float(weather_by_hour[hour]["TEMP"]),
            "RAIN": float(weather_by_hour[hour]["RAIN"]),
            "DELIV_TEMP": float(weather_by_hour[hour]["TEMP"]),
            "DELIV_RAIN": float(weather_by_hour[hour]["RAIN"]),
        })

        X_row = pd.DataFrame([row])
        X_row = X_row.drop(columns=[c for c in DROP_COLS if c in X_row.columns], errors="ignore")

        required = get_required_feature_names_from_preprocess(bundle["preprocess"])
        X_row_aligned = align_row_to_required_columns(X_row, required)

        pred = float(predict_with_bundle(bundle, X_row_aligned)[0])
        if USE_LOG_TARGET:
            pred = float(np.expm1(pred))

        rows.append({
            "TA_YMD": str(date_yyyymmdd),
            "DAY": int(day_code),
            "HOUR": int(hour),
            "BEST_MODEL": best_name,
            "TEMP": round(float(row["TEMP"]), 2), 
            "RAIN": float(row["RAIN"]),
            "PRED_UNIT": int(round(pred)),
            "KMA_BASE_USED": str(hourly_weather.loc[0, "base_used"]),
        })

    return pd.DataFrame(rows).sort_values("HOUR").reset_index(drop=True)

# =========================
# ✅ 핵심: 사용자 DONG 입력 버전 (요청한 흐름)
# - DONG으로 nx,ny 자동 추출
# - 그 nx,ny로 예측 수행
# =========================
def predict_date_all_hours_with_weather_by_dong(
    date_yyyymmdd: str,
    dong: str,
    service_key: str = SERVICE_KEY
) -> pd.DataFrame:
    nx, ny = find_grid_by_dong(dong)
    df_pred = predict_date_all_hours_with_weather(
        date_yyyymmdd=date_yyyymmdd,
        nx=nx,
        ny=ny,
        service_key=service_key
    )
    df_pred.insert(1, "DONG", str(dong))
#     df_pred.insert(2, "nx", int(nx))
#     df_pred.insert(3, "ny", int(ny))
    return df_pred

# =========================
# 사용 예시
# =========================
if __name__ == "__main__":
    # 1) 동의 특정 시간대(항목요약) 기온만 뽑기
#     temp = get_temperature_by_dong("20251231", dong="정자2동", slot_hour=5)
#     print("TEMP(slot_hour=5):", temp, "°C")

    # 2) 동 입력으로 1~10 시간대 매출(UNIT) 예측
    df_pred = predict_date_all_hours_with_weather_by_dong(
        date_yyyymmdd="20251231",
        dong="정자2동"
    )
    print(df_pred)


TEMP(slot_hour=5): -4.0 °C
     TA_YMD  DONG  DAY  HOUR BEST_MODEL  TEMP  RAIN  PRED_UNIT  KMA_BASE_USED
0  20251231  정자2동    3     1        XGB -5.43   0.0      38545  20251230-1100
1  20251231  정자2동    3     2        XGB -8.00   0.0      11827  20251230-1100
2  20251231  정자2동    3     3        XGB -7.00   0.0      16934  20251230-1100
3  20251231  정자2동    3     4  EXTRATREE -5.50   0.0      25851  20251230-1100
4  20251231  정자2동    3     5  EXTRATREE -4.00   0.0      24799  20251230-1100
5  20251231  정자2동    3     6        XGB -4.00   0.0      20502  20251230-1100
6  20251231  정자2동    3     7  EXTRATREE -5.50   0.0      32290  20251230-1100
7  20251231  정자2동    3     8  EXTRATREE -6.50   0.0      38535  20251230-1100
8  20251231  정자2동    3     9        XGB -8.00   0.0      34702  20251230-1100
9  20251231  정자2동    3    10        XGB -9.00   0.0      51965  20251230-1100


## 매출 데이터 입력 후 예측

# 6. 휴게소, 빵집 추가

In [ ]:
import os
import json
import time
import warnings
import joblib
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import ExtraTreesRegressor

warnings.filterwarnings("ignore")

# =============================
# XGB 사용 가능 여부
# =============================
try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except ImportError:
    HAS_XGB = False


# =============================
# Config
# =============================
BASE_PATH   = "data/수원시 한식 데이터백업.csv"     
DELIV_PATH  = "data/수원시 배달 데이터백업.csv"
BAKERY_PATH = "data/수원시 빵집 데이터백업.csv"
REST_PATH   = "data/수원시 휴게소 데이터백업.csv"

SAVE_DIR = "data/model_amt_fast"
TEST_RATIO = 0.2
RANDOM_STATE = 42

TARGET_COL = "AMT"

# Lag / Rolling (AMT 기준)
LAGS = [1, 7, 14]
ROLL_WINDOWS = [7, 14]

# OneHot 제한
OHE_MIN_FREQUENCY = 20
OHE_MAX_CATEGORIES = 200

# XGB (속도 최적)
XGB_PARAMS = dict(
    n_estimators=1800,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="reg:squarederror",
    eval_metric="mae",
    tree_method="hist",
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

ET_PARAMS = dict(
    n_estimators=600,
    min_samples_leaf=2,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

DROP_COLS = ["AMT", "DATE"]


# =============================
# Utils
# =============================
def evaluate(y, p):
    return (
        mean_absolute_error(y, p),
        mean_squared_error(y, p, squared=False),
        r2_score(y, p),
    )


def safe_ohe():
    try:
        return OneHotEncoder(
            handle_unknown="ignore",
            min_frequency=OHE_MIN_FREQUENCY,
            max_categories=OHE_MAX_CATEGORIES,
            sparse_output=True
        )
    except:
        return OneHotEncoder(handle_unknown="ignore")


def build_preprocess(X):
    cat = [c for c in X.columns if X[c].dtype == "object"]
    num = [c for c in X.columns if c not in cat]

    return ColumnTransformer([
        ("num", SimpleImputer(strategy="median"), num),
        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("ohe", safe_ohe())
        ]), cat)
    ])


def prefix(df, name, keys):
    out = df.copy()
    for c in out.columns:
        if c not in keys:
            out.rename(columns={c: f"{name}_{c}"}, inplace=True)
    return out


# =============================
# Load & Merge
# =============================
merge_keys = ["TA_YMD", "DONG", "HOUR", "DAY"]

base   = pd.read_csv(BASE_PATH)
deliv  = prefix(pd.read_csv(DELIV_PATH),  "DELIV",  merge_keys)
bakery = prefix(pd.read_csv(BAKERY_PATH), "BAKERY", merge_keys)
rest   = prefix(pd.read_csv(REST_PATH),   "REST",   merge_keys)

df = (
    base
    .merge(deliv, on=merge_keys, how="left")
    .merge(bakery, on=merge_keys, how="left")
    .merge(rest, on=merge_keys, how="left")
)

# =============================
# Date features
# =============================
df["DATE"] = pd.to_datetime(df["TA_YMD"].astype(str), format="%Y%m%d")
df["YEAR"] = df["DATE"].dt.year
df["MONTH"] = df["DATE"].dt.month
df["WEEKOFYEAR"] = df["DATE"].dt.isocalendar().week.astype(int)

# =============================
# Lag / Rolling (AMT)
# =============================
df = df.sort_values(["DONG", "HOUR", "DATE"])
grp = df.groupby(["DONG", "HOUR"])

for l in LAGS:
    df[f"AMT_LAG_{l}"] = grp[TARGET_COL].shift(l)

for w in ROLL_WINDOWS:
    df[f"AMT_ROLL_MEAN_{w}"] = grp[TARGET_COL].shift(1).rolling(w, min_periods=3).mean()
    df[f"AMT_ROLL_STD_{w}"]  = grp[TARGET_COL].shift(1).rolling(w, min_periods=3).std()

# =============================
# Target / Feature split
# =============================
y = df[TARGET_COL]
X = df.drop(columns=[c for c in DROP_COLS if c in df.columns])

# 시간 순 분할
order = np.argsort(df["DATE"].values)
X, y = X.iloc[order], y.iloc[order]

split = int(len(X) * (1 - TEST_RATIO))
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

# =============================
# Encode
# =============================
pre = build_preprocess(X_train)
X_tr = pre.fit_transform(X_train)
X_te = pre.transform(X_test)

# =============================
# Train
# =============================
candidates = []

if HAS_XGB:
    xgb = XGBRegressor(**XGB_PARAMS)
    xgb.fit(
        X_tr, y_train,
        eval_set=[(X_te, y_test)],
        verbose=False,
        early_stopping_rounds=100
    )
    p = xgb.predict(X_te)
    candidates.append(("XGB", xgb, *evaluate(y_test, p)))

et = ExtraTreesRegressor(**ET_PARAMS)
et.fit(X_tr, y_train)
p = et.predict(X_te)
candidates.append(("ET", et, *evaluate(y_test, p)))

best = sorted(candidates, key=lambda x: x[2])[0]

# =============================
# Save
# =============================
os.makedirs(SAVE_DIR, exist_ok=True)
joblib.dump(
    {"model": best[1], "preprocess": pre, "name": best[0]},
    os.path.join(SAVE_DIR, "best_model_amt.joblib")
)

print(f"""
[DONE]
MODEL = {best[0]}
MAE   = {best[2]:,.2f}
RMSE  = {best[3]:,.2f}
R2    = {best[4]:.4f}
""")


# 7. 하루매출, 동별 매출을 중요시한 모델

In [3]:
# -*- coding: utf-8 -*-
import os
import json
import time
import warnings
import joblib
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.base import clone
from xgboost import XGBRegressor

warnings.filterwarnings("ignore")

# =============================
# TensorFlow 사용 가능 여부 체크 (HOUR=1 딥러닝용)
# =============================
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    HAS_TF = True
except Exception:
    HAS_TF = False

# =============================
# Config
# =============================
HANSIK_PATH = "data/수원시 한식 데이터백업.csv"

SAVE_DIR = "data/model_amt"
MAX_HOURS = list(range(1, 11))    # 1~10
TEST_RATIO = 0.2                  # 마지막 20% 테스트(시간순)
RANDOM_STATE = 42

OHE_MIN_FREQUENCY = 20
OHE_MAX_CATEGORIES = 200

LAGS = [1, 7, 14]
ROLL_WINDOWS = [7, 14]

XGB_PARAMS = dict(
    n_estimators=4000,
    learning_rate=0.03,
    max_depth=7,
    min_child_weight=2,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    reg_alpha=0.0,
    objective="reg:absoluteerror",
    tree_method="hist",
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

DL_CFG = dict(
    epochs=60,
    batch_size=1024,
    patience=8,
    lr=1e-3,
    hidden_units=[128, 64],
    dropout=0.10
)

DROP_COLS = ["DATE", "AMT", "CNT", "UNIT"]


# =============================
# Utils
# =============================
def evaluate_reg(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    return float(mae), float(rmse), float(r2)

def safe_onehot_encoder():
    try:
        return OneHotEncoder(
            handle_unknown="ignore",
            min_frequency=OHE_MIN_FREQUENCY,
            max_categories=OHE_MAX_CATEGORIES
        )
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore")

def build_preprocess_xgb(X: pd.DataFrame):
    cat_cols = [c for c in X.columns if X[c].dtype == "object"]
    num_cols = [c for c in X.columns if c not in cat_cols]

    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ])
    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", safe_onehot_encoder())
    ])

    return ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_cols),
            ("cat", categorical_transformer, cat_cols)
        ],
        remainder="drop"
    )

def build_preprocess_dl_numeric(X: pd.DataFrame):
    num_cols = [c for c in X.columns if X[c].dtype != "object"]
    pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    return pipe, num_cols

def seed_everything(seed=42):
    np.random.seed(seed)
    try:
        import random
        random.seed(seed)
    except Exception:
        pass
    if HAS_TF:
        tf.random.set_seed(seed)

seed_everything(RANDOM_STATE)

# =============================
# 1) Load
# =============================
df = pd.read_csv(HANSIK_PATH)

required = {"TA_YMD", "DONG", "HOUR", "DAY", "AMT"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"[한식 데이터] 필수 컬럼이 없습니다: {missing}")

# =============================
# 2) Date features
# =============================
df["TA_YMD"] = df["TA_YMD"].astype(str)
df["DATE"] = pd.to_datetime(df["TA_YMD"], format="%Y%m%d", errors="coerce")
df = df[df["DATE"].notna()].copy()

df["YEAR"] = df["DATE"].dt.year
df["MONTH"] = df["DATE"].dt.month
df["DAY_OF_MONTH"] = df["DATE"].dt.day
df["WEEKOFYEAR"] = df["DATE"].dt.isocalendar().week.astype(int)

# =============================
# 3) Lag/Rolling features for AMT (no leakage if time-split)
# =============================
df = df.sort_values(["DONG", "HOUR", "DATE"]).reset_index(drop=True)
grp = df.groupby(["DONG", "HOUR"], sort=False)

for lag in LAGS:
    df[f"AMT_LAG_{lag}"] = grp["AMT"].shift(lag)

for w in ROLL_WINDOWS:
    df[f"AMT_ROLL_MEAN_{w}"] = grp["AMT"].shift(1).rolling(window=w, min_periods=max(2, w // 3)).mean()
    df[f"AMT_ROLL_STD_{w}"]  = grp["AMT"].shift(1).rolling(window=w, min_periods=max(2, w // 3)).std()

df["AMT_LAG_1_MISSING"] = df["AMT_LAG_1"].isna().astype(int)

# =============================
# 3-1) DONG 중요도 강화 (에러 수정 버전)
# - transform 기반: df 인덱스에 1:1로 안전하게 붙음
# - 누수 방지: shift(1) 이후 expanding
# =============================
def expanding_mean_shift1(s: pd.Series, min_periods: int):
    return s.shift(1).expanding(min_periods=min_periods).mean()

def expanding_median_shift1(s: pd.Series, min_periods: int):
    return s.shift(1).expanding(min_periods=min_periods).median()

df["DONGHOUR_AMT_EXP_MEAN"] = grp["AMT"].transform(lambda s: expanding_mean_shift1(s, min_periods=5))
df["DONGHOUR_AMT_EXP_MED"]  = grp["AMT"].transform(lambda s: expanding_median_shift1(s, min_periods=5))

grp_d = df.groupby("DONG", sort=False)
df["DONG_AMT_EXP_MEAN"] = grp_d["AMT"].transform(lambda s: expanding_mean_shift1(s, min_periods=10))

# =============================
# Target / Features
# =============================
y_all = df["AMT"].copy()
X_all = df.drop(columns=[c for c in DROP_COLS if c in df.columns], errors="ignore")

# =============================
# 4) Train per HOUR
# =============================
os.makedirs(SAVE_DIR, exist_ok=True)

hours = sorted(pd.unique(df["HOUR"].dropna()))
hours = [int(h) for h in hours if int(h) in MAX_HOURS]

print(f"[INFO] TF 사용 여부(HOUR=1 딥러닝): {HAS_TF}")
print(f"[INFO] 학습 시간대: {hours}")
print(f"[INFO] 저장 폴더: {SAVE_DIR}")

all_results = []
t0 = time.time()

def train_hour1_deep_learning(X_train, y_train, X_test, y_test, hour_dir):
    if not HAS_TF:
        raise RuntimeError("TensorFlow가 없어 딥러닝 학습 불가")

    # train/val split
    ntr = len(X_train)
    split2 = int(ntr * 0.9)
    X_tr, X_val = X_train.iloc[:split2].copy(), X_train.iloc[split2:].copy()
    y_tr, y_val = y_train.iloc[:split2].copy(), y_train.iloc[split2:].copy()

    # DONG integer encoding
    dong_train = X_tr["DONG"].astype(str).values
    dong_val   = X_val["DONG"].astype(str).values
    dong_test  = X_test["DONG"].astype(str).values

    uniq = pd.Index(pd.unique(dong_train))
    dong2id = {k: i+1 for i, k in enumerate(uniq)}  # 0: OOV

    def map_dong(arr):
        return np.array([dong2id.get(x, 0) for x in arr], dtype=np.int32)

    dong_tr_id = map_dong(dong_train)
    dong_val_id = map_dong(dong_val)
    dong_test_id = map_dong(dong_test)

    # numeric preprocess
    X_tr_num = X_tr.drop(columns=["DONG"], errors="ignore")
    X_val_num = X_val.drop(columns=["DONG"], errors="ignore")
    X_test_num = X_test.drop(columns=["DONG"], errors="ignore")

    num_pipe, num_cols = build_preprocess_dl_numeric(X_tr_num)
    X_tr_num_s = num_pipe.fit_transform(X_tr_num[num_cols])
    X_val_num_s = num_pipe.transform(X_val_num[num_cols])
    X_test_num_s = num_pipe.transform(X_test_num[num_cols])

    # model
    vocab_size = int(len(dong2id) + 1)
    emb_dim = int(min(32, max(8, round(np.sqrt(vocab_size)))))

    inp_dong = keras.Input(shape=(1,), dtype="int32", name="dong_id")
    x_dong = layers.Embedding(input_dim=vocab_size+1, output_dim=emb_dim, name="dong_emb")(inp_dong)
    x_dong = layers.Flatten()(x_dong)

    inp_num = keras.Input(shape=(X_tr_num_s.shape[1],), dtype="float32", name="num")
    x = layers.Concatenate()([x_dong, inp_num])

    for u in DL_CFG["hidden_units"]:
        x = layers.Dense(u, activation="relu")(x)
        x = layers.Dropout(DL_CFG["dropout"])(x)

    out = layers.Dense(1, name="amt")(x)

    model = keras.Model(inputs=[inp_dong, inp_num], outputs=out)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=DL_CFG["lr"]),
        loss="mae",
        metrics=[keras.metrics.MeanAbsoluteError(name="mae")]
    )

    cb = [
        keras.callbacks.EarlyStopping(monitor="val_mae", patience=DL_CFG["patience"], restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(monitor="val_mae", factor=0.7, patience=3, min_lr=1e-5),
    ]

    model.fit(
        {"dong_id": dong_tr_id, "num": X_tr_num_s},
        y_tr.values,
        validation_data=({"dong_id": dong_val_id, "num": X_val_num_s}, y_val.values),
        epochs=DL_CFG["epochs"],
        batch_size=DL_CFG["batch_size"],
        verbose=0,
        callbacks=cb
    )

    pred = model.predict({"dong_id": dong_test_id, "num": X_test_num_s}, verbose=0).reshape(-1)
    mae, rmse, r2 = evaluate_reg(y_test.values, pred)

    # save
    model_path = os.path.join(hour_dir, "dl_model.keras")
    model.save(model_path)

    pre_path = os.path.join(hour_dir, "dl_preprocess.joblib")
    joblib.dump({"dong2id": dong2id, "num_pipe": num_pipe, "num_cols": num_cols}, pre_path)

    bundle = {"model_name": "DL_MLP_DONG_EMB", "model_path": model_path, "preprocess_path": pre_path, "type": "DL"}
    return bundle, mae, rmse, r2

def predict_hour1_dl(bundle, X_new: pd.DataFrame):
    if not HAS_TF:
        raise RuntimeError("TensorFlow가 없어 딥러닝 예측 불가")

    from tensorflow import keras
    model = keras.models.load_model(bundle["model_path"])
    pre = joblib.load(bundle["preprocess_path"])
    dong2id = pre["dong2id"]
    num_pipe = pre["num_pipe"]
    num_cols = pre["num_cols"]

    dong_arr = X_new["DONG"].astype(str).values
    dong_id = np.array([dong2id.get(x, 0) for x in dong_arr], dtype=np.int32)

    X_num = X_new.drop(columns=["DONG"], errors="ignore")
    X_num_s = num_pipe.transform(X_num[num_cols])

    pred = model.predict({"dong_id": dong_id, "num": X_num_s}, verbose=0).reshape(-1)
    return pred

print("\n[DEBUG] >>> 학습 루프 시작")
for h in hours:
    idx = (df["HOUR"].astype(int) == h)
    X_h = X_all.loc[idx].copy()
    y_h = y_all.loc[idx].copy()
    d_h = df.loc[idx, "DATE"].copy()

    if len(X_h) < 800:
        print(f"\n[SKIP] HOUR={h:02d}: 데이터가 너무 적음 (n={len(X_h)})")
        continue

    # 시간순 split
    order = np.argsort(d_h.values)
    X_h = X_h.iloc[order].reset_index(drop=True)
    y_h = y_h.iloc[order].reset_index(drop=True)

    n = len(X_h)
    split = int(n * (1 - TEST_RATIO))
    X_train, X_test = X_h.iloc[:split], X_h.iloc[split:]
    y_train, y_test = y_h.iloc[:split], y_h.iloc[split:]

    print(f"\n===== HOUR={h:02d} | train={len(X_train)} test={len(X_test)} =====")

    hour_dir = os.path.join(SAVE_DIR, f"HOUR_{h:02d}")
    os.makedirs(hour_dir, exist_ok=True)

    # HOUR=1: 딥러닝
    if h == 1 and HAS_TF:
        try:
            bundle, mae, rmse, r2 = train_hour1_deep_learning(X_train, y_train, X_test, y_test, hour_dir)
            best_path = os.path.join(hour_dir, "best_model.joblib")
            joblib.dump(bundle, best_path)

            meta = {
                "HOUR": int(h),
                "BEST_MODEL": bundle["model_name"],
                "TYPE": bundle["type"],
                "MAE": float(mae),
                "RMSE": float(rmse),
                "R2": float(r2),
                "TRAIN_SIZE": int(len(X_train)),
                "TEST_SIZE": int(len(X_test)),
                "FEATURE_COLS": list(X_all.columns),
                "DROP_COLS": DROP_COLS,
                "LAGS": LAGS,
                "ROLL_WINDOWS": ROLL_WINDOWS,
                "DL_CFG": DL_CFG,
                "HAS_TF": bool(HAS_TF),
            }
            with open(os.path.join(hour_dir, "meta.json"), "w", encoding="utf-8") as f:
                json.dump(meta, f, ensure_ascii=False, indent=2)

            print(f"- DL(H1)    | MAE={mae:,.3f} RMSE={rmse:,.3f} R2={r2:,.4f}")
            print(f"✅ BEST => {bundle['model_name']} 저장: {best_path}")

            all_results.append({"HOUR": int(h), "BEST_MODEL": bundle["model_name"], "TYPE": bundle["type"],
                                "MAE": float(mae), "RMSE": float(rmse), "R2": float(r2), "SAVE_PATH": best_path})
            continue
        except Exception as e:
            print(f"⚠️ HOUR=1 딥러닝 실패 -> XGB로 대체. error={e}")

    # XGB (2~10 + HOUR=1 fallback)
    split2 = int(len(X_train) * 0.9)
    X_tr, X_val = X_train.iloc[:split2], X_train.iloc[split2:]
    y_tr, y_val = y_train.iloc[:split2], y_train.iloc[split2:]

    preprocess = build_preprocess_xgb(X_train)
    pre = clone(preprocess)

    X_tr_enc = pre.fit_transform(X_tr)
    X_val_enc = pre.transform(X_val)
    X_test_enc = pre.transform(X_test)

    xgb_params = dict(XGB_PARAMS)
    try:
        model_xgb = XGBRegressor(**xgb_params)
        model_xgb.fit(X_tr_enc, y_tr, eval_set=[(X_val_enc, y_val)], verbose=False, early_stopping_rounds=200)
    except Exception:
        xgb_params["objective"] = "reg:squarederror"
        model_xgb = XGBRegressor(**xgb_params)
        model_xgb.fit(X_tr_enc, y_tr, eval_set=[(X_val_enc, y_val)], verbose=False, early_stopping_rounds=200)

    pred = model_xgb.predict(X_test_enc)
    mae, rmse, r2 = evaluate_reg(y_test, pred)
    print(f"- XGB       | MAE={mae:,.3f} RMSE={rmse:,.3f} R2={r2:,.4f}")

    best_bundle = {"preprocess": pre, "model": model_xgb, "model_name": "XGB", "type": "XGB"}
    best_path = os.path.join(hour_dir, "best_model.joblib")
    joblib.dump(best_bundle, best_path)

    meta = {
        "HOUR": int(h),
        "BEST_MODEL": "XGB",
        "TYPE": "XGB",
        "MAE": float(mae),
        "RMSE": float(rmse),
        "R2": float(r2),
        "TRAIN_SIZE": int(len(X_train)),
        "TEST_SIZE": int(len(X_test)),
        "FEATURE_COLS": list(X_all.columns),
        "DROP_COLS": DROP_COLS,
        "LAGS": LAGS,
        "ROLL_WINDOWS": ROLL_WINDOWS,
        "HAS_TF": bool(HAS_TF),
        "XGB_PARAMS": XGB_PARAMS,
    }
    with open(os.path.join(hour_dir, "meta.json"), "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    print(f"✅ BEST => XGB | MAE={mae:,.3f} 저장: {best_path}")

    all_results.append({"HOUR": int(h), "BEST_MODEL": "XGB", "TYPE": "XGB",
                        "MAE": float(mae), "RMSE": float(rmse), "R2": float(r2), "SAVE_PATH": best_path})

# 결과 저장
res_df = pd.DataFrame(all_results).sort_values(["HOUR"])
res_path = os.path.join(SAVE_DIR, "results_by_hour.csv")
res_df.to_csv(res_path, index=False, encoding="utf-8-sig")

print("\n[DONE] 학습 완료")
print(f"- 결과표 저장: {res_path}")
print(f"- 모델 저장 폴더: {SAVE_DIR}")
print(f"- 총 소요(초): {time.time() - t0:,.1f}")


# =============================
# 5) 로드/예측 헬퍼
# =============================
def load_best_bundle(hour: int):
    hour_dir = os.path.join(SAVE_DIR, f"HOUR_{hour:02d}")
    path = os.path.join(hour_dir, "best_model.joblib")
    return joblib.load(path)

def predict_with_bundle(bundle: dict, X_new: pd.DataFrame) -> np.ndarray:
    if bundle.get("type") == "DL":
        return predict_hour1_dl(bundle, X_new)
    pre = bundle["preprocess"]
    model = bundle["model"]
    X_enc = pre.transform(X_new)
    return model.predict(X_enc)


[INFO] TF 사용 여부(HOUR=1 딥러닝): True
[INFO] 학습 시간대: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[INFO] 저장 폴더: data/model_amt

[DEBUG] >>> 학습 루프 시작

===== HOUR=01 | train=33935 test=8484 =====
- DL(H1)    | MAE=1,422,978.290 RMSE=5,048,622.482 R2=0.3067
✅ BEST => DL_MLP_DONG_EMB 저장: data/model_amt\HOUR_01\best_model.joblib

===== HOUR=02 | train=33998 test=8500 =====
- XGB       | MAE=248,367.541 RMSE=953,382.150 R2=0.4135
✅ BEST => XGB | MAE=248,367.541 저장: data/model_amt\HOUR_02\best_model.joblib

===== HOUR=03 | train=35715 test=8929 =====
- XGB       | MAE=645,458.140 RMSE=2,432,873.733 R2=0.3735
✅ BEST => XGB | MAE=645,458.140 저장: data/model_amt\HOUR_03\best_model.joblib

===== HOUR=04 | train=36360 test=9091 =====
- XGB       | MAE=1,821,287.913 RMSE=5,831,440.797 R2=0.8165
✅ BEST => XGB | MAE=1,821,287.913 저장: data/model_amt\HOUR_04\best_model.joblib

===== HOUR=05 | train=36408 test=9102 =====
- XGB       | MAE=2,171,725.638 RMSE=6,673,013.365 R2=0.8493
✅ BEST => XGB | MAE=2,171,725.638 저장: da

## mae값 낮추기

In [4]:
%%time
import os
import json
import time
import warnings
import joblib
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.base import clone
from xgboost import XGBRegressor

warnings.filterwarnings("ignore")

# =============================
# TensorFlow 사용 가능 여부 체크 (HOUR=1 딥러닝용)
# =============================
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    HAS_TF = True
except Exception:
    HAS_TF = False


# =============================
# Config
# =============================
HANSIK_PATH = "data/수원시 한식 데이터백업.csv"

SAVE_DIR = "data/model_amt"
MAX_HOURS = list(range(1, 11))    # 1~10
TEST_RATIO = 0.2
RANDOM_STATE = 42

# ✅ AMT 꼬리/아웃라이어 많으면 MAE 개선에 도움되는 경우가 많음
USE_LOG_TARGET = True            # <-- (추가) log1p 타겟 사용

# 원핫 폭발 방지
OHE_MIN_FREQUENCY = 10           # <-- (완화) 더 많은 카테고리 반영(동 중요도 ↑)
OHE_MAX_CATEGORIES = 500

# ✅ 피처 강화(조금만)
LAGS = [1, 7, 14, 28]            # <-- (추가)
ROLL_WINDOWS = [7, 14, 30, 60]   # <-- (추가)

# ✅ 30분 정도 쓸 수 있게 XGB를 강하게(early stopping으로 자동 컷)
# - learning_rate ↓, n_estimators ↑, early_stopping ↑
# - grow_policy=lossguide + max_leaves로 더 강하게(데이터 크면 효율 좋음)
XGB_PARAMS = dict(
    n_estimators=30000,          # <-- 크게
    learning_rate=0.01,          # <-- 작게
    max_depth=0,                # <-- lossguide에서 depth 대신 leaves로
    max_leaves=256,             # <-- (추가) 모델 용량
    grow_policy="lossguide",    # <-- (추가)
    min_child_weight=1,
    subsample=0.85,
    colsample_bytree=0.85,
    reg_lambda=2.0,
    reg_alpha=0.1,
    objective="reg:squarederror",   # 안정적으로 + eval_metric=mae로 MAE 최적화
    eval_metric="mae",
    tree_method="hist",
    max_bin=512,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

EARLY_STOPPING_ROUNDS = 1500     # <-- (증가) 더 오래 학습

# ✅ 딥러닝(HOUR=1)도 조금 더 오래/강하게
DL_CFG = dict(
    epochs=250,                 # <-- 증가
    batch_size=512,             # <-- 조금 작게(성능↑ 가능)
    patience=25,                # <-- 증가
    lr=5e-4,                    # <-- 조금 낮게
    hidden_units=[256, 128, 64],# <-- 증가
    dropout=0.15
)

# 누수 위험 제거
DROP_COLS = ["DATE", "AMT", "CNT", "UNIT"]


# =============================
# Utils
# =============================
def evaluate_reg(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    return float(mae), float(rmse), float(r2)

def safe_onehot_encoder():
    try:
        return OneHotEncoder(
            handle_unknown="ignore",
            min_frequency=OHE_MIN_FREQUENCY,
            max_categories=OHE_MAX_CATEGORIES
        )
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore")

def build_preprocess_xgb(X: pd.DataFrame):
    cat_cols = [c for c in X.columns if X[c].dtype == "object"]
    num_cols = [c for c in X.columns if c not in cat_cols]

    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ])
    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", safe_onehot_encoder())
    ])

    return ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_cols),
            ("cat", categorical_transformer, cat_cols)
        ],
        remainder="drop"
    )

def build_preprocess_dl_numeric(X: pd.DataFrame):
    num_cols = [c for c in X.columns if X[c].dtype != "object"]
    pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    return pipe, num_cols

def seed_everything(seed=42):
    np.random.seed(seed)
    try:
        import random
        random.seed(seed)
    except Exception:
        pass
    if HAS_TF:
        tf.random.set_seed(seed)

seed_everything(RANDOM_STATE)

def y_transform(y: pd.Series):
    """학습용 타겟 변환"""
    if USE_LOG_TARGET:
        return np.log1p(np.clip(y.values, 0, None))
    return y.values.astype(float)

def y_inverse(pred: np.ndarray):
    """예측값 역변환"""
    if USE_LOG_TARGET:
        return np.expm1(pred)
    return pred


# =============================
# 1) Load
# =============================
df = pd.read_csv(HANSIK_PATH)

required = {"TA_YMD", "DONG", "HOUR", "DAY", "AMT"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"[한식 데이터] 필수 컬럼이 없습니다: {missing}")

# =============================
# 2) Date features
# =============================
df["TA_YMD"] = df["TA_YMD"].astype(str)
df["DATE"] = pd.to_datetime(df["TA_YMD"], format="%Y%m%d", errors="coerce")
df = df[df["DATE"].notna()].copy()

df["YEAR"] = df["DATE"].dt.year
df["MONTH"] = df["DATE"].dt.month
df["DAY_OF_MONTH"] = df["DATE"].dt.day
df["WEEKOFYEAR"] = df["DATE"].dt.isocalendar().week.astype(int)

# =============================
# 3) Lag/Rolling features for AMT (no leakage)
# =============================
df = df.sort_values(["DONG", "HOUR", "DATE"]).reset_index(drop=True)
grp = df.groupby(["DONG", "HOUR"], sort=False)

for lag in LAGS:
    df[f"AMT_LAG_{lag}"] = grp["AMT"].shift(lag)

for w in ROLL_WINDOWS:
    df[f"AMT_ROLL_MEAN_{w}"] = grp["AMT"].shift(1).rolling(window=w, min_periods=max(2, w // 3)).mean()
    df[f"AMT_ROLL_STD_{w}"]  = grp["AMT"].shift(1).rolling(window=w, min_periods=max(2, w // 3)).std()

df["AMT_LAG_1_MISSING"] = df["AMT_LAG_1"].isna().astype(int)

# =============================
# 3-1) DONG 중요도 강화(누수 방지, IndexError 방지: transform 사용)
# =============================
def expanding_mean_shift1(s: pd.Series, min_periods: int):
    return s.shift(1).expanding(min_periods=min_periods).mean()

def expanding_median_shift1(s: pd.Series, min_periods: int):
    return s.shift(1).expanding(min_periods=min_periods).median()

def expanding_std_shift1(s: pd.Series, min_periods: int):
    return s.shift(1).expanding(min_periods=min_periods).std()

df["DONGHOUR_AMT_EXP_MEAN"] = grp["AMT"].transform(lambda s: expanding_mean_shift1(s, 5))
df["DONGHOUR_AMT_EXP_MED"]  = grp["AMT"].transform(lambda s: expanding_median_shift1(s, 5))
df["DONGHOUR_AMT_EXP_STD"]  = grp["AMT"].transform(lambda s: expanding_std_shift1(s, 5))  # <-- (추가)

grp_d = df.groupby("DONG", sort=False)
df["DONG_AMT_EXP_MEAN"] = grp_d["AMT"].transform(lambda s: expanding_mean_shift1(s, 10))

# =============================
# Target / Features
# =============================
y_all = df["AMT"].copy()
X_all = df.drop(columns=[c for c in DROP_COLS if c in df.columns], errors="ignore")


# =============================
# 4) Train per HOUR
# =============================
os.makedirs(SAVE_DIR, exist_ok=True)

hours = sorted(pd.unique(df["HOUR"].dropna()))
hours = [int(h) for h in hours if int(h) in MAX_HOURS]

print(f"[INFO] TF 사용 여부(HOUR=1 딥러닝): {HAS_TF}")
print(f"[INFO] log 타겟 사용: {USE_LOG_TARGET}")
print(f"[INFO] 학습 시간대: {hours}")
print(f"[INFO] 저장 폴더: {SAVE_DIR}")

all_results = []
t0 = time.time()

def train_hour1_deep_learning(X_train, y_train_raw, X_test, y_test_raw, hour_dir):
    if not HAS_TF:
        raise RuntimeError("TensorFlow가 없어 딥러닝 학습 불가")

    # train/val split (시간순)
    ntr = len(X_train)
    split2 = int(ntr * 0.9)
    X_tr, X_val = X_train.iloc[:split2].copy(), X_train.iloc[split2:].copy()
    y_tr_raw, y_val_raw = y_train_raw.iloc[:split2].copy(), y_train_raw.iloc[split2:].copy()

    # y transform
    y_tr = y_transform(y_tr_raw)
    y_val = y_transform(y_val_raw)

    # DONG integer encoding
    dong_train = X_tr["DONG"].astype(str).values
    dong_val   = X_val["DONG"].astype(str).values
    dong_test  = X_test["DONG"].astype(str).values

    uniq = pd.Index(pd.unique(dong_train))
    dong2id = {k: i+1 for i, k in enumerate(uniq)}  # 0 OOV

    def map_dong(arr):
        return np.array([dong2id.get(x, 0) for x in arr], dtype=np.int32)

    dong_tr_id  = map_dong(dong_train)
    dong_val_id = map_dong(dong_val)
    dong_test_id= map_dong(dong_test)

    # numeric preprocess
    X_tr_num = X_tr.drop(columns=["DONG"], errors="ignore")
    X_val_num = X_val.drop(columns=["DONG"], errors="ignore")
    X_test_num = X_test.drop(columns=["DONG"], errors="ignore")

    num_pipe, num_cols = build_preprocess_dl_numeric(X_tr_num)
    X_tr_num_s = num_pipe.fit_transform(X_tr_num[num_cols])
    X_val_num_s = num_pipe.transform(X_val_num[num_cols])
    X_test_num_s = num_pipe.transform(X_test_num[num_cols])

    # model
    vocab_size = int(len(dong2id) + 1)
    emb_dim = int(min(64, max(12, round(np.sqrt(vocab_size) * 2))))  # <-- 약간 확대

    inp_dong = keras.Input(shape=(1,), dtype="int32", name="dong_id")
    x_dong = layers.Embedding(input_dim=vocab_size+1, output_dim=emb_dim, name="dong_emb")(inp_dong)
    x_dong = layers.Flatten()(x_dong)

    inp_num = keras.Input(shape=(X_tr_num_s.shape[1],), dtype="float32", name="num")
    x = layers.Concatenate()([x_dong, inp_num])

    x = layers.LayerNormalization()(x)  # <-- (추가) 안정성/성능 개선

    for u in DL_CFG["hidden_units"]:
        x = layers.Dense(u, activation="relu")(x)
        x = layers.Dropout(DL_CFG["dropout"])(x)

    out = layers.Dense(1, name="amt")(x)

    model = keras.Model(inputs=[inp_dong, inp_num], outputs=out)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=DL_CFG["lr"]),
        loss="mae",
        metrics=[keras.metrics.MeanAbsoluteError(name="mae")]
    )

    cb = [
        keras.callbacks.EarlyStopping(monitor="val_mae", patience=DL_CFG["patience"], restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(monitor="val_mae", factor=0.7, patience=5, min_lr=1e-5),
    ]

    model.fit(
        {"dong_id": dong_tr_id, "num": X_tr_num_s},
        y_tr,
        validation_data=({"dong_id": dong_val_id, "num": X_val_num_s}, y_val),
        epochs=DL_CFG["epochs"],
        batch_size=DL_CFG["batch_size"],
        verbose=0,
        callbacks=cb
    )

    # predict + inverse
    pred_t = model.predict({"dong_id": dong_test_id, "num": X_test_num_s}, verbose=0).reshape(-1)
    pred = y_inverse(pred_t)
    pred = np.clip(pred, 0, None)

    mae, rmse, r2 = evaluate_reg(y_test_raw.values, pred)

    # save
    model_path = os.path.join(hour_dir, "dl_model.keras")
    model.save(model_path)

    pre_path = os.path.join(hour_dir, "dl_preprocess.joblib")
    joblib.dump({"dong2id": dong2id, "num_pipe": num_pipe, "num_cols": num_cols}, pre_path)

    bundle = {"model_name": "DL_MLP_DONG_EMB", "model_path": model_path, "preprocess_path": pre_path, "type": "DL"}
    return bundle, mae, rmse, r2

def predict_hour1_dl(bundle, X_new: pd.DataFrame):
    if not HAS_TF:
        raise RuntimeError("TensorFlow가 없어 딥러닝 예측 불가")

    from tensorflow import keras
    model = keras.models.load_model(bundle["model_path"])
    pre = joblib.load(bundle["preprocess_path"])
    dong2id = pre["dong2id"]
    num_pipe = pre["num_pipe"]
    num_cols = pre["num_cols"]

    dong_arr = X_new["DONG"].astype(str).values
    dong_id = np.array([dong2id.get(x, 0) for x in dong_arr], dtype=np.int32)

    X_num = X_new.drop(columns=["DONG"], errors="ignore")
    X_num_s = num_pipe.transform(X_num[num_cols])

    pred_t = model.predict({"dong_id": dong_id, "num": X_num_s}, verbose=0).reshape(-1)
    pred = y_inverse(pred_t)
    return np.clip(pred, 0, None)

print("\n[DEBUG] >>> 학습 루프 시작")
for h in hours:
    idx = (df["HOUR"].astype(int) == h)
    X_h = X_all.loc[idx].copy()
    y_h_raw = y_all.loc[idx].copy()
    d_h = df.loc[idx, "DATE"].copy()

    if len(X_h) < 800:
        print(f"\n[SKIP] HOUR={h:02d}: 데이터가 너무 적음 (n={len(X_h)})")
        continue

    # 시간순 split
    order = np.argsort(d_h.values)
    X_h = X_h.iloc[order].reset_index(drop=True)
    y_h_raw = y_h_raw.iloc[order].reset_index(drop=True)

    n = len(X_h)
    split = int(n * (1 - TEST_RATIO))
    X_train, X_test = X_h.iloc[:split], X_h.iloc[split:]
    y_train_raw, y_test_raw = y_h_raw.iloc[:split], y_h_raw.iloc[split:]

    print(f"\n===== HOUR={h:02d} | train={len(X_train)} test={len(X_test)} =====")

    hour_dir = os.path.join(SAVE_DIR, f"HOUR_{h:02d}")
    os.makedirs(hour_dir, exist_ok=True)

    # HOUR=1: 딥러닝
    if h == 1 and HAS_TF:
        try:
            bundle, mae, rmse, r2 = train_hour1_deep_learning(X_train, y_train_raw, X_test, y_test_raw, hour_dir)
            best_path = os.path.join(hour_dir, "best_model.joblib")
            joblib.dump(bundle, best_path)

            meta = {
                "HOUR": int(h),
                "BEST_MODEL": bundle["model_name"],
                "TYPE": bundle["type"],
                "MAE": float(mae),
                "RMSE": float(rmse),
                "R2": float(r2),
                "TRAIN_SIZE": int(len(X_train)),
                "TEST_SIZE": int(len(X_test)),
                "FEATURE_COLS": list(X_all.columns),
                "DROP_COLS": DROP_COLS,
                "LAGS": LAGS,
                "ROLL_WINDOWS": ROLL_WINDOWS,
                "DL_CFG": DL_CFG,
                "USE_LOG_TARGET": USE_LOG_TARGET,
                "HAS_TF": bool(HAS_TF),
            }
            with open(os.path.join(hour_dir, "meta.json"), "w", encoding="utf-8") as f:
                json.dump(meta, f, ensure_ascii=False, indent=2)

            print(f"- DL(H1)    | MAE={mae:,.3f} RMSE={rmse:,.3f} R2={r2:,.4f}")
            print(f"✅ BEST => {bundle['model_name']} 저장: {best_path}")

            all_results.append({"HOUR": int(h), "BEST_MODEL": bundle["model_name"], "TYPE": bundle["type"],
                                "MAE": float(mae), "RMSE": float(rmse), "R2": float(r2), "SAVE_PATH": best_path})
            continue
        except Exception as e:
            print(f"⚠️ HOUR=1 딥러닝 실패 -> XGB로 대체. error={e}")

    # XGB (2~10 + HOUR=1 fallback)
    # train 마지막 10%를 valid로 early stopping
    split2 = int(len(X_train) * 0.9)
    X_tr, X_val = X_train.iloc[:split2], X_train.iloc[split2:]
    y_tr_raw, y_val_raw = y_train_raw.iloc[:split2], y_train_raw.iloc[split2:]

    # y transform
    y_tr = y_transform(y_tr_raw)
    y_val = y_transform(y_val_raw)

    preprocess = build_preprocess_xgb(X_train)
    pre = clone(preprocess)

    X_tr_enc = pre.fit_transform(X_tr)
    X_val_enc = pre.transform(X_val)
    X_test_enc = pre.transform(X_test)

    model_xgb = XGBRegressor(**XGB_PARAMS)
    model_xgb.fit(
        X_tr_enc, y_tr,
        eval_set=[(X_val_enc, y_val)],
        verbose=False,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS
    )

    pred_t = model_xgb.predict(X_test_enc)
    pred = y_inverse(pred_t)
    pred = np.clip(pred, 0, None)

    mae, rmse, r2 = evaluate_reg(y_test_raw.values, pred)
    print(f"- XGB       | MAE={mae:,.3f} RMSE={rmse:,.3f} R2={r2:,.4f}  (best_iter={getattr(model_xgb, 'best_iteration', None)})")

    best_bundle = {"preprocess": pre, "model": model_xgb, "model_name": "XGB", "type": "XGB"}
    best_path = os.path.join(hour_dir, "best_model.joblib")
    joblib.dump(best_bundle, best_path)

    meta = {
        "HOUR": int(h),
        "BEST_MODEL": "XGB",
        "TYPE": "XGB",
        "MAE": float(mae),
        "RMSE": float(rmse),
        "R2": float(r2),
        "TRAIN_SIZE": int(len(X_train)),
        "TEST_SIZE": int(len(X_test)),
        "FEATURE_COLS": list(X_all.columns),
        "DROP_COLS": DROP_COLS,
        "LAGS": LAGS,
        "ROLL_WINDOWS": ROLL_WINDOWS,
        "USE_LOG_TARGET": USE_LOG_TARGET,
        "HAS_TF": bool(HAS_TF),
        "XGB_PARAMS": XGB_PARAMS,
        "EARLY_STOPPING_ROUNDS": EARLY_STOPPING_ROUNDS,
    }
    with open(os.path.join(hour_dir, "meta.json"), "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    print(f"✅ BEST => XGB | MAE={mae:,.3f} 저장: {best_path}")

    all_results.append({"HOUR": int(h), "BEST_MODEL": "XGB", "TYPE": "XGB",
                        "MAE": float(mae), "RMSE": float(rmse), "R2": float(r2), "SAVE_PATH": best_path})

# 결과 저장
res_df = pd.DataFrame(all_results).sort_values(["HOUR"])
res_path = os.path.join(SAVE_DIR, "results_by_hour.csv")
res_df.to_csv(res_path, index=False, encoding="utf-8-sig")

print("\n[DONE] 학습 완료")
print(f"- 결과표 저장: {res_path}")
print(f"- 모델 저장 폴더: {SAVE_DIR}")
print(f"- 총 소요(초): {time.time() - t0:,.1f}")

# =============================
# 5) 로드/예측 헬퍼
# =============================
def load_best_bundle(hour: int):
    hour_dir = os.path.join(SAVE_DIR, f"HOUR_{hour:02d}")
    path = os.path.join(hour_dir, "best_model.joblib")
    return joblib.load(path)

def predict_with_bundle(bundle: dict, X_new: pd.DataFrame) -> np.ndarray:
    if bundle.get("type") == "DL":
        return predict_hour1_dl(bundle, X_new)
    pre = bundle["preprocess"]
    model = bundle["model"]
    X_enc = pre.transform(X_new)
    pred_t = model.predict(X_enc)
    pred = y_inverse(pred_t)
    return np.clip(pred, 0, None)


[INFO] TF 사용 여부(HOUR=1 딥러닝): True
[INFO] log 타겟 사용: True
[INFO] 학습 시간대: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[INFO] 저장 폴더: data/model_amt

[DEBUG] >>> 학습 루프 시작

===== HOUR=01 | train=33935 test=8484 =====
- DL(H1)    | MAE=953,988.580 RMSE=2,532,482.998 R2=0.8255
✅ BEST => DL_MLP_DONG_EMB 저장: data/model_amt\HOUR_01\best_model.joblib

===== HOUR=02 | train=33998 test=8500 =====
- XGB       | MAE=228,565.487 RMSE=926,586.537 R2=0.4460  (best_iter=730)
✅ BEST => XGB | MAE=228,565.487 저장: data/model_amt\HOUR_02\best_model.joblib

===== HOUR=03 | train=35715 test=8929 =====
- XGB       | MAE=593,664.265 RMSE=2,432,275.543 R2=0.3738  (best_iter=534)
✅ BEST => XGB | MAE=593,664.265 저장: data/model_amt\HOUR_03\best_model.joblib

===== HOUR=04 | train=36360 test=9091 =====
- XGB       | MAE=1,648,562.293 RMSE=5,964,550.638 R2=0.8080  (best_iter=1003)
✅ BEST => XGB | MAE=1,648,562.293 저장: data/model_amt\HOUR_04\best_model.joblib

===== HOUR=05 | train=36408 test=9102 =====
- XGB       | MAE=1,796,913.

## mae값 낮추고 성능 올리기

In [9]:
%%time
import os
import time
import warnings
import joblib
import numpy as np
import pandas as pd

from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from xgboost import XGBRegressor

warnings.filterwarnings("ignore")


# =========================
# Config
# =========================
@dataclass
class CFG:
    DATA_PATH: str = "data/수원시 한식 동별 데이터백업.csv"          
    SAVE_ROOT: str = "data/model_amt_xgb"          
    USE_LOG_TARGET: bool = True                  

    # 시간대 1~10 (항목요약)
    HOURS: Tuple[int, ...] = tuple(range(1, 11))
    TEST_RATIO: float = 0.2                       # 마지막 20% 테스트(시간순)

    # 동별 성능 핵심: dong+hour 기준 lag/rolling
    LAGS: Tuple[int, ...] = (1, 2, 3, 7)
    ROLL_WINDOWS: Tuple[int, ...] = (3, 7)

    RANDOM_STATE: int = 42
    TOTAL_TIME_BUDGET_SEC: int = 1200             # 20분

    # 진행 로그 출력
    VERBOSE: bool = True


# =========================
# Utils
# =========================
def log(msg: str, cfg: CFG):
    if cfg.VERBOSE:
        print(msg, flush=True)

def to_dt(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.replace("-", "", regex=False)
    return pd.to_datetime(s, format="%Y%m%d", errors="coerce")

def time_split_last_ratio(df: pd.DataFrame, dt_col: str, test_ratio: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
    df = df.sort_values(dt_col).reset_index(drop=True)
    n = len(df)
    cut = int(np.floor(n * (1.0 - test_ratio)))
    cut = max(1, min(cut, n - 1))
    return df.iloc[:cut].copy(), df.iloc[cut:].copy()

def add_lag_rolling(df: pd.DataFrame, target: str, group_cols: List[str], dt_col: str,
                    lags: Tuple[int, ...], wins: Tuple[int, ...]) -> pd.DataFrame:
    """
    dong+hour 기준 lag/rolling → 동별 예측 성능에 가장 크게 기여
    """
    df = df.sort_values(group_cols + [dt_col]).reset_index(drop=True)
    g = df.groupby(group_cols, dropna=False)[target]

    for k in lags:
        df[f"{target}_lag{k}"] = g.shift(k)

    for w in wins:
        s = g.shift(1)
        df[f"{target}_rollmean{w}"] = s.rolling(w).mean()
        df[f"{target}_rollmax{w}"] = s.rolling(w).max()
        df[f"{target}_rollmin{w}"] = s.rolling(w).min()

    return df

def build_preprocess(X: pd.DataFrame) -> ColumnTransformer:
    # 숫자/범주 자동 분리 (DONG 같은 문자열은 cat)
    cat_cols = [c for c in X.columns if X[c].dtype == "object" or str(X[c].dtype).startswith("string")]
    num_cols = [c for c in X.columns if c not in cat_cols]

    num_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ])
    cat_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore", sparse=True)),
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", num_pipe, num_cols),
            ("cat", cat_pipe, cat_cols),
        ],
        remainder="drop",
        sparse_threshold=0.3
    )
    return pre


def make_param_candidates(seed: int) -> List[Dict]:
    """
    20분 안에 끝내기 위해 랜덤서치 대신 '잘 먹는' 후보 파라미터 8개 정도만 평가.
    (각 후보는 early stopping으로 best_iter로 컷)
    """
    rng = np.random.RandomState(seed)
    cands = []

    # base families (fast + strong)
    for max_depth in [4, 6, 8]:
        for lr in [0.03, 0.05]:
            cands.append(dict(
                max_depth=max_depth,
                learning_rate=lr,
                subsample=0.85,
                colsample_bytree=0.85,
                min_child_weight=1.0,
                reg_alpha=0.0,
                reg_lambda=1.0,
                gamma=0.0,
            ))

    # 조금 더 regularized / robust
    cands += [
        dict(max_depth=6, learning_rate=0.03, subsample=0.75, colsample_bytree=0.75, min_child_weight=3.0, reg_alpha=0.0, reg_lambda=5.0, gamma=0.0),
        dict(max_depth=8, learning_rate=0.02, subsample=0.75, colsample_bytree=0.85, min_child_weight=5.0, reg_alpha=1e-3, reg_lambda=10.0, gamma=0.0),
    ]

    # shuffle a bit
    rng.shuffle(cands)
    return cands[:8]


def fit_xgb_one(
    X_train: pd.DataFrame,
    y_train: np.ndarray,
    X_valid: pd.DataFrame,
    y_valid: np.ndarray,
    preprocess: ColumnTransformer,
    params: Dict,
    seed: int
):
    """
    preprocess → transform → XGB early stopping
    """
    Xtr = preprocess.fit_transform(X_train, y_train)
    Xva = preprocess.transform(X_valid)

    model = XGBRegressor(
        objective="reg:absoluteerror",
        n_estimators=8000,                 # 크게 두고 early stopping으로 best_iter 결정
        tree_method="hist",
        random_state=seed,
        n_jobs=-1,

        # tuned params
        **params
    )

    model.fit(
        Xtr, y_train,
        eval_set=[(Xva, y_valid)],
        eval_metric="mae",
        early_stopping_rounds=200,
        verbose=False
    )
    return preprocess, model


def eval_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    return {"mae": float(mae), "rmse": float(rmse), "r2": float(r2)}


# =========================
# Main Training Loop (HOUR 1~10)
# =========================
def train_all_hours_xgb(cfg: CFG):
    t_all0 = time.time()
    os.makedirs(cfg.SAVE_ROOT, exist_ok=True)

    # ---------- Load ----------
    df = pd.read_csv(cfg.DATA_PATH)
    required = {"TA_YMD", "DONG", "HOUR", "DAY", "AMT", "CNT", "UNIT", "TEMP", "RAIN"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"데이터 컬럼 누락: {missing}. 현재 컬럼={list(df.columns)}")

    df["TA_YMD"] = df["TA_YMD"].astype(str)
    df["_dt"] = to_dt(df["TA_YMD"])
    if df["_dt"].isna().all():
        raise ValueError("TA_YMD 날짜 파싱 실패. 예: 20251231 형태여야 합니다.")

    # ---------- Info ----------
    log(f"[INFO] TF 사용 여부(HOUR=1 딥러닝): False", cfg)
    log(f"[INFO] log 타겟 사용: {cfg.USE_LOG_TARGET}", cfg)
    log(f"[INFO] 학습 시간대: {list(cfg.HOURS)}", cfg)
    log(f"[INFO] 저장 폴더: {cfg.SAVE_ROOT}\n", cfg)
    log("[DEBUG] >>> 학습 루프 시작\n", cfg)

    # 결과표
    results_rows = []

    # 시간 예산을 시간대별로 쪼개기(대략 균등)
    per_hour_budget = max(60, int(cfg.TOTAL_TIME_BUDGET_SEC / len(cfg.HOURS)))

    # ---------- loop each hour ----------
    for hour in cfg.HOURS:
        t0 = time.time()

        # hour filter
        d = df[df["HOUR"] == hour].copy()
        d = d.sort_values("_dt").reset_index(drop=True)

        # 시간기반 split
        train_df, test_df = time_split_last_ratio(d, "_dt", cfg.TEST_RATIO)

        log(f"===== HOUR={hour:02d} | train={len(train_df)} test={len(test_df)} =====", cfg)

        # dong+hour 기준 lag/rolling (동별 성능)
        # hour는 고정이지만 group_cols에 넣으면 동별 시계열로 계산되므로 DONG만 넣어도 동일.
        train_df2 = add_lag_rolling(train_df, "AMT", group_cols=["DONG"], dt_col="_dt",
                                    lags=cfg.LAGS, wins=cfg.ROLL_WINDOWS)
        test_df2 = add_lag_rolling(test_df, "AMT", group_cols=["DONG"], dt_col="_dt",
                                   lags=cfg.LAGS, wins=cfg.ROLL_WINDOWS)

        # feature set (UNIT/CNT는 타겟과 유사해 누수 가능성이 커서 기본은 제외)
        # 필요하면 아래 drop에서 빼도 됨.
        drop_cols = ["AMT", "_dt"]  # target + helper
        # 동별 중요: DONG 포함
        feature_cols = [c for c in train_df2.columns if c not in drop_cols]

        # (권장) 누수/의미 애매한 컬럼 제거
        # - UNIT/CNT가 "실제 결과"에 가까운 값이면 예측 시점에 없으므로 빼야 함
        for leak in ["UNIT", "CNT"]:
            if leak in feature_cols:
                feature_cols.remove(leak)

        X_train_full = train_df2[feature_cols].copy()
        y_train_full = train_df2["AMT"].astype(float).values

        X_test = test_df2[feature_cols].copy()
        y_test = test_df2["AMT"].astype(float).values

        # train 내부 valid 분리(마지막 15%를 valid)
        ntr = len(X_train_full)
        vcut = int(ntr * 0.85)
        vcut = max(1, min(vcut, ntr - 1))

        X_train = X_train_full.iloc[:vcut].copy()
        y_train = y_train_full[:vcut]

        X_valid = X_train_full.iloc[vcut:].copy()
        y_valid = y_train_full[vcut:]

        # log target
        if cfg.USE_LOG_TARGET:
            y_train_fit = np.log1p(y_train)
            y_valid_fit = np.log1p(y_valid)
        else:
            y_train_fit = y_train
            y_valid_fit = y_valid

        # 후보 파라미터들
        cands = make_param_candidates(cfg.RANDOM_STATE + hour)

        best = None
        best_mae = float("inf")
        best_info = None

        # preprocess는 후보마다 새로 만드는 게 안전(컬럼 동일하지만 fit 상태 분리)
        # 시간 예산 체크하면서 후보 평가
        for i, p in enumerate(cands, start=1):
            if time.time() - t0 > per_hour_budget:
                log(f"  [WARN] 시간 예산 초과로 후보 평가 조기 종료 (i={i-1}/{len(cands)})", cfg)
                break

            pre = build_preprocess(X_train_full)
            pre_fitted, model = fit_xgb_one(
                X_train=X_train,
                y_train=y_train_fit,
                X_valid=X_valid,
                y_valid=y_valid_fit,
                preprocess=pre,
                params=p,
                seed=cfg.RANDOM_STATE + hour
            )

            # test 평가
            Xte_enc = pre_fitted.transform(X_test)
            pred = model.predict(Xte_enc)
            if cfg.USE_LOG_TARGET:
                pred = np.expm1(pred)

            m = eval_metrics(y_test, pred)
            best_iter = getattr(model, "best_iteration", None)
            if best_iter is None:
                best_iter = getattr(model, "best_ntree_limit", None)

            log(f"  - XGB cand#{i:<2} | MAE={m['mae']:,.3f} RMSE={m['rmse']:,.3f} R2={m['r2']:.4f} (best_iter={best_iter})", cfg)

            if m["mae"] < best_mae:
                best_mae = m["mae"]
                best = {"preprocess": pre_fitted, "model": model}
                best_info = {"params": p, "metrics": m, "best_iter": best_iter}

        if best is None:
            raise RuntimeError(f"HOUR={hour:02d}에서 모델 학습이 실패했습니다(후보가 모두 스킵됨).")

        # 저장
        hour_dir = os.path.join(cfg.SAVE_ROOT, f"HOUR_{hour:02d}")
        os.makedirs(hour_dir, exist_ok=True)
        save_path = os.path.join(hour_dir, "best_model.joblib")

        joblib.dump(
            {
                "type": "xgb_only",
                "hour": hour,
                "use_log_target": cfg.USE_LOG_TARGET,
                "feature_cols": feature_cols,
                "preprocess": best["preprocess"],
                "model": best["model"],
                "best_params": best_info["params"],
                "metrics_test": best_info["metrics"],
                "best_iter": best_info["best_iter"],
            },
            save_path
        )

        log(f"✅ BEST => XGB | MAE={best_info['metrics']['mae']:,.3f} 저장: {save_path}\n", cfg)

        results_rows.append({
            "HOUR": hour,
            "BEST_MODEL": "XGB",
            "MAE": best_info["metrics"]["mae"],
            "RMSE": best_info["metrics"]["rmse"],
            "R2": best_info["metrics"]["r2"],
            "BEST_ITER": best_info["best_iter"],
            "MODEL_PATH": save_path,
        })

    # 결과표 저장
    res_df = pd.DataFrame(results_rows).sort_values("HOUR").reset_index(drop=True)
    res_path = os.path.join(cfg.SAVE_ROOT, "results_by_hour.csv")
    res_df.to_csv(res_path, index=False, encoding="utf-8-sig")

    total_sec = time.time() - t_all0
    log("[DONE] 학습 완료", cfg)
    log(f"- 결과표 저장: {res_path}", cfg)
    log(f"- 모델 저장 폴더: {cfg.SAVE_ROOT}", cfg)
    log(f"- 총 소요(초): {total_sec:,.1f}", cfg)


# =========================
# (선택) 예측 함수: 저장된 모델로 예측
# =========================
def load_hour_model(save_root: str, hour: int) -> dict:
    path = os.path.join(save_root, f"HOUR_{hour:02d}", "best_model.joblib")
    if not os.path.exists(path):
        raise FileNotFoundError(f"모델이 없습니다: {path}")
    return joblib.load(path)

def predict_amt_one_hour(model_obj: dict, X_new: pd.DataFrame) -> np.ndarray:
    """
    X_new는 feature_cols를 포함하는 DF여야 함 (DONG, TA_YMD, DAY, HOUR, TEMP, RAIN 등)
    """
    pre = model_obj["preprocess"]
    mdl = model_obj["model"]
    feat_cols = model_obj["feature_cols"]

    X_use = X_new[feat_cols].copy()
    X_enc = pre.transform(X_use)
    pred = mdl.predict(X_enc)
    if model_obj.get("use_log_target", False):
        pred = np.expm1(pred)
    return pred


if __name__ == "__main__":
    cfg = CFG(
        DATA_PATH="data/수원시 한식 동별 데이터백업.csv",  
        SAVE_ROOT="data/model_amt_xgb",
        USE_LOG_TARGET=True,
        TOTAL_TIME_BUDGET_SEC=1200,         # 20분
        VERBOSE=True
    )
    train_all_hours_xgb(cfg)


[INFO] TF 사용 여부(HOUR=1 딥러닝): False
[INFO] log 타겟 사용: True
[INFO] 학습 시간대: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[INFO] 저장 폴더: data/model_amt_xgb

[DEBUG] >>> 학습 루프 시작

===== HOUR=01 | train=33935 test=8484 =====
  - XGB cand#1  | MAE=822,312.078 RMSE=2,196,652.914 R2=0.8687 (best_iter=26)
  - XGB cand#2  | MAE=912,250.451 RMSE=2,852,431.266 R2=0.7787 (best_iter=8)
  - XGB cand#3  | MAE=905,422.539 RMSE=2,711,424.367 R2=0.8000 (best_iter=6)
  - XGB cand#4  | MAE=836,157.679 RMSE=2,120,713.352 R2=0.8777 (best_iter=19)
  - XGB cand#5  | MAE=822,312.078 RMSE=2,196,652.914 R2=0.8687 (best_iter=26)
  - XGB cand#6  | MAE=909,725.543 RMSE=3,254,827.312 R2=0.7118 (best_iter=26)
  - XGB cand#7  | MAE=909,725.543 RMSE=3,254,827.312 R2=0.7118 (best_iter=26)
  - XGB cand#8  | MAE=905,422.539 RMSE=2,711,424.367 R2=0.8000 (best_iter=6)
✅ BEST => XGB | MAE=822,312.078 저장: data/model_amt_xgb\HOUR_01\best_model.joblib

===== HOUR=02 | train=33998 test=8500 =====
  - XGB cand#1  | MAE=261,688.021 RMSE=972,487.8

# 8. claude 수정

In [3]:
# -*- coding: utf-8 -*-
"""
시간대별 AMT/CNT 예측 (수원시 한식 동별 데이터) - 개선 버전
- 입력: 날짜(TA_YMD), DONG, (기상청 API로 가져온) 시간대별 TEMP/RAIN
- 출력: 시간대별 AMT, CNT

개선사항:
1. 피처 엔지니어링 강화 (온도 구간, 계절, 주말 등)
2. 하이퍼파라미터 최적화
3. 이상치 처리
4. R² 스코어 평가 추가
5. 모델별 최적화
"""

import os
import math
import joblib
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from dotenv import load_dotenv

# =========================
# A) 설정
# =========================
load_dotenv()
SERVICE_KEY = os.getenv('RAIN_ID')
DATA_CSV = "data/수원시 한식 동별 데이터백업.csv"
MODEL_DIR = "data/models_hourly_amt_cnt2"
os.makedirs(MODEL_DIR, exist_ok=True)

# ---- 기상청 단기예보(getVilageFcst) 엔드포인트 ----
KMA_VILAGE_URL = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
BASE_TIMES = ["2300", "2000", "1700", "1400", "1100", "0800", "0500", "0200"]

# HOUR(01~10) 구간 대표 시각
HOUR_SLOT_TO_REP_TIME = {
    1: "0300", 2: "0800", 3: "1000", 4: "1200", 5: "1400",
    6: "1600", 7: "1800", 8: "2000", 9: "2200", 10: "2300",
}

# ---- DONG -> (nx, ny) 격자 좌표 매핑 ----
DONG_GRID_CSV = "data/dong_grid.csv"
DONG_TO_GRID = {}


# =========================
# B) 유틸: 요일 계산 (1~7, 월=1 ... 일=7)
# =========================
def ymd_to_day_1to7(yyyymmdd: str) -> int:
    dt = datetime.strptime(str(yyyymmdd), "%Y%m%d")
    return dt.weekday() + 1


# =========================
# C) 피처 엔지니어링 함수 추가
# =========================
def create_features(df):
    """
    추가 피처 생성:
    - TEMP_BIN: 온도 구간 (분석에서 발견한 최적 구간 활용)
    - IS_WEEKEND: 주말 여부
    - SEASON: 계절
    - TEMP_SQUARED: 온도 제곱 (비선형 관계)
    - IS_HOT: 무더운 날씨 (27도 이상)
    - IS_COLD: 추운 날씨 (-7~1도)
    """
    df = df.copy()
    
    # 온도 구간 (분석 결과 기반)
    df['TEMP_BIN'] = pd.cut(df['TEMP'], 
                            bins=[-100, -7, 1, 10, 18, 27, 100],
                            labels=['very_cold', 'cold', 'cool', 'pleasant', 'warm', 'hot'])
    df['TEMP_BIN'] = df['TEMP_BIN'].astype(str)
    
    # 주말 여부 (토=6, 일=7)
    df['IS_WEEKEND'] = (df['DAY'] >= 6).astype(int)
    
    # 계절 (월 추출 필요)
    df['MONTH'] = df['TA_YMD'].astype(str).str[4:6].astype(int)
    df['SEASON'] = df['MONTH'].apply(lambda m: 
        'spring' if 3 <= m <= 5 else
        'summer' if 6 <= m <= 8 else
        'fall' if 9 <= m <= 11 else 'winter'
    )
    
    # 온도 제곱 (역U자 패턴 반영)
    df['TEMP_SQUARED'] = df['TEMP'] ** 2
    
    # 특정 온도 구간 플래그
    df['IS_HOT'] = (df['TEMP'] >= 27).astype(int)
    df['IS_COLD'] = ((df['TEMP'] >= -7) & (df['TEMP'] <= 1)).astype(int)
    df['IS_OPTIMAL'] = ((df['TEMP'] >= 10) & (df['TEMP'] <= 14)).astype(int)
    
    # 강수량 로그 변환 (0은 작은 값으로 대체)
    df['RAIN_LOG'] = np.log1p(df['RAIN'])
    
    # 시간대 특성
    df['IS_LUNCH'] = (df['HOUR'] == 4).astype(int)  # 점심시간대
    df['IS_DINNER'] = (df['HOUR'].isin([7, 8])).astype(int)  # 저녁시간대
    df['IS_DAWN'] = (df['HOUR'] == 1).astype(int)  # 새벽시간대
    
    return df


def remove_outliers(df, target_col, threshold_percentile=99):
    """이상치 제거"""
    threshold = df[target_col].quantile(threshold_percentile / 100)
    return df[df[target_col] <= threshold]


# =========================
# D) 기상청 API 함수들 (원본 유지)
# =========================
def pick_base_datetime_kst(now_kst: datetime) -> tuple[str, str]:
    today = now_kst.strftime("%Y%m%d")
    hhmm = now_kst.strftime("%H%M")
    for bt in BASE_TIMES:
        if hhmm >= bt:
            return today, bt
    yday = (now_kst - timedelta(days=1)).strftime("%Y%m%d")
    return yday, "2300"


def fetch_vilage_fcst_items(service_key: str, base_date: str, base_time: str, nx: int, ny: int,
                            num_of_rows: int = 2000) -> list[dict]:
    params = {
        "serviceKey": service_key,
        "pageNo": 1,
        "numOfRows": num_of_rows,
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": nx,
        "ny": ny,
    }
    r = requests.get(KMA_VILAGE_URL, params=params, timeout=20)
    r.raise_for_status()
    data = r.json()
    items = data["response"]["body"]["items"]["item"]
    return items


def parse_pcp_to_float(pcp_val) -> float:
    if pcp_val is None:
        return 0.0
    s = str(pcp_val).strip()
    if s in ("강수없음", "없음", ""):
        return 0.0
    s = s.replace("mm", "").replace("미만", "").strip()
    try:
        return float(s)
    except:
        return 0.0


def get_temp_rain_for_datetime(service_key: str, dong: str, target_yyyymmdd: str, target_hhmm: str) -> tuple[float, float]:
    if not DONG_TO_GRID:
        if os.path.exists(DONG_GRID_CSV):
            g = pd.read_csv(DONG_GRID_CSV)
            for _, row in g.iterrows():
                DONG_TO_GRID[str(row["DONG"])] = (int(row["nx"]), int(row["ny"]))
        else:
            raise ValueError(f"[필수] {DONG_GRID_CSV}를 만들거나 DONG_TO_GRID dict를 채워주세요.")

    if dong not in DONG_TO_GRID:
        raise ValueError(f"DONG_TO_GRID에 '{dong}'의 nx/ny가 없습니다.")

    nx, ny = DONG_TO_GRID[dong]
    now_kst = datetime.now(timezone(timedelta(hours=9)))
    base_date, base_time = pick_base_datetime_kst(now_kst)
    items = fetch_vilage_fcst_items(service_key, base_date, base_time, nx, ny)

    tmp = None
    pcp = None
    for it in items:
        if it.get("fcstDate") == target_yyyymmdd and it.get("fcstTime") == target_hhmm:
            cat = it.get("category")
            if cat == "TMP":
                tmp = float(it.get("fcstValue"))
            elif cat == "PCP":
                pcp = parse_pcp_to_float(it.get("fcstValue"))

    if tmp is None:
        tmp = 0.0
    if pcp is None:
        pcp = 0.0
    return tmp, pcp


def get_hourly_weather_features(service_key: str, dong: str, yyyymmdd: str) -> pd.DataFrame:
    day_1to7 = ymd_to_day_1to7(yyyymmdd)
    rows = []
    for hour_slot in range(1, 11):
        rep_time = HOUR_SLOT_TO_REP_TIME[hour_slot]
        temp, rain = get_temp_rain_for_datetime(service_key, dong, yyyymmdd, rep_time)
        rows.append({
            "TA_YMD": int(yyyymmdd),
            "DONG": dong,
            "HOUR": hour_slot,
            "DAY": day_1to7,
            "TEMP": float(temp),
            "RAIN": float(rain),
        })
    df = pd.DataFrame(rows)
    return create_features(df)


# =========================
# E) 개선된 전처리 파이프라인
# =========================
def build_preprocess():
    """개선된 전처리 - 더 많은 피처 활용"""
    cat_cols = ["DONG", "TEMP_BIN", "SEASON"]
    num_cols = ["DAY", "TEMP", "RAIN", "TEMP_SQUARED", "RAIN_LOG",
                "IS_WEEKEND", "IS_HOT", "IS_COLD", "IS_OPTIMAL",
                "IS_LUNCH", "IS_DINNER", "IS_DAWN"]
    
    pre = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
            ("num", StandardScaler(), num_cols),  # 스케일링 추가
        ],
        remainder="drop",
    )
    return pre


# =========================
# F) 개선된 모델 학습
# =========================
def train_and_save_models(random_state: int = 42):
    df = pd.read_csv(DATA_CSV)
    
    # 타입 정리
    df["DONG"] = df["DONG"].astype(str)
    df["HOUR"] = df["HOUR"].astype(int)
    df["DAY"] = df["DAY"].astype(int)
    df["TEMP"] = df["TEMP"].astype(float)
    df["RAIN"] = df["RAIN"].astype(float)
    
    # 피처 생성
    df = create_features(df)
    
    # 이상치 제거 (AMT 기준 상위 1%)
    print(f"원본 데이터: {len(df):,}건")
    df = remove_outliers(df, "AMT", threshold_percentile=99)
    print(f"이상치 제거 후: {len(df):,}건")
    
    y_cols = ["AMT", "CNT"]
    metrics = []

    for h in range(1, 11):
        print(f"\n{'='*60}")
        print(f"HOUR {h:02d} 모델 학습 중...")
        print(f"{'='*60}")
        
        sub = df[df["HOUR"] == h].copy()
        sub = sub.dropna(subset=["DONG", "DAY", "TEMP", "RAIN", "AMT", "CNT"])
        
        # 피처 선택
        feature_cols = ["DONG", "DAY", "TEMP", "RAIN", "TEMP_BIN", "SEASON",
                       "TEMP_SQUARED", "RAIN_LOG", "IS_WEEKEND", "IS_HOT", 
                       "IS_COLD", "IS_OPTIMAL", "IS_LUNCH", "IS_DINNER", "IS_DAWN"]
        
        X = sub[feature_cols]
        y = sub[y_cols]
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state, shuffle=True
        )
        
        pre = build_preprocess()
        
        if h == 1:
            # HOUR=1: 개선된 MLPRegressor
            base_model = MLPRegressor(
                hidden_layer_sizes=(256, 128, 64),  # 더 깊은 네트워크
                activation="relu",
                solver="adam",
                alpha=1e-4,
                batch_size=128,  # 배치 크기 감소
                learning_rate="adaptive",  # 적응형 학습률
                learning_rate_init=5e-4,
                max_iter=500,  # 더 많은 에포크
                random_state=42,
                early_stopping=True,
                validation_fraction=0.1,
                n_iter_no_change=20,
                tol=1e-5,
                verbose=False,
            )
        else:
            # HOUR=2~10: 최적화된 XGBRegressor
            base_model = XGBRegressor(
                n_estimators=800,  # 트리 개수
                learning_rate=0.04,  # 학습률
                max_depth=7,  # 깊이
                min_child_weight=3,
                subsample=0.8,
                colsample_bytree=0.8,
                reg_alpha=0.1,  # L1 정규화
                reg_lambda=1.5,  # L2 정규화
                gamma=0.1,  # 최소 손실 감소
                objective="reg:squarederror",
                random_state=random_state,
                tree_method="hist",
                n_jobs=-1,
            )
        
        model = MultiOutputRegressor(base_model)
        
        pipe = Pipeline([
            ("pre", pre),
            ("model", model),
        ])
        
        # 모델 학습
        pipe.fit(X_train, y_train)
        
        # 예측 및 평가
        pred = pipe.predict(X_test)
        
        # AMT 평가
        mae_amt = mean_absolute_error(y_test["AMT"].values, pred[:, 0])
        r2_amt = r2_score(y_test["AMT"].values, pred[:, 0])
        rmse_amt = np.sqrt(mean_squared_error(y_test["AMT"].values, pred[:, 0]))
        
        # CNT 평가
        mae_cnt = mean_absolute_error(y_test["CNT"].values, pred[:, 1])
        r2_cnt = r2_score(y_test["CNT"].values, pred[:, 1])
        rmse_cnt = np.sqrt(mean_squared_error(y_test["CNT"].values, pred[:, 1]))
        
        # 모델 저장
        model_path = os.path.join(MODEL_DIR, f"hour_{h:02d}_amt_cnt.joblib")
        joblib.dump(pipe, model_path)
        
        metrics.append({
            "HOUR": h,
            "MAE_AMT": mae_amt,
            "R2_AMT": r2_amt,
            "RMSE_AMT": rmse_amt,
            "MAE_CNT": mae_cnt,
            "R2_CNT": r2_cnt,
            "RMSE_CNT": rmse_cnt,
            "model_path": model_path,
            "model_type": "MLP(Deep)" if h == 1 else "XGB",
            "n_train": len(X_train),
            "n_test": len(X_test),
        })
        
        print(f"\n[HOUR {h:02d}] 저장 완료 -> {model_path}")
        print(f"  AMT - MAE: {mae_amt:,.1f} | R²: {r2_amt:.4f} | RMSE: {rmse_amt:,.1f}")
        print(f"  CNT - MAE: {mae_cnt:,.2f} | R²: {r2_cnt:.4f} | RMSE: {rmse_cnt:,.2f}")
    
    metrics_df = pd.DataFrame(metrics).sort_values("HOUR")
    metrics_df.to_csv(os.path.join(MODEL_DIR, "metrics_summary.csv"), index=False, encoding="utf-8-sig")
    
    print("\n" + "="*60)
    print("전체 모델 성능 요약")
    print("="*60)
    print(f"\nAMT 평균 성능:")
    print(f"  - MAE: {metrics_df['MAE_AMT'].mean():,.1f}")
    print(f"  - R²: {metrics_df['R2_AMT'].mean():.4f}")
    print(f"\nCNT 평균 성능:")
    print(f"  - MAE: {metrics_df['MAE_CNT'].mean():.2f}")
    print(f"  - R²: {metrics_df['R2_CNT'].mean():.4f}")
    print("\n=== metrics_summary.csv 저장 완료 ===\n")
    
    return metrics_df


# =========================
# G) 예측 함수 (원본 유지하되 피처 추가)
# =========================
def load_models():
    models = {}
    for h in range(1, 11):
        p = os.path.join(MODEL_DIR, f"hour_{h:02d}_amt_cnt.joblib")
        if not os.path.exists(p):
            raise FileNotFoundError(f"모델 파일이 없습니다: {p}")
        models[h] = joblib.load(p)
    return models


def predict_day(yyyymmdd: str, dong: str) -> pd.DataFrame:
    # 기상청에서 시간대별 TEMP/RAIN 구성 (피처 포함)
    feat_df = get_hourly_weather_features(SERVICE_KEY, dong, yyyymmdd)
    
    models = load_models()
    
    outs = []
    for h in range(1, 11):
        row_data = feat_df[feat_df["HOUR"] == h]
        
        # 필요한 피처만 선택
        feature_cols = ["DONG", "DAY", "TEMP", "RAIN", "TEMP_BIN", "SEASON",
                       "TEMP_SQUARED", "RAIN_LOG", "IS_WEEKEND", "IS_HOT", 
                       "IS_COLD", "IS_OPTIMAL", "IS_LUNCH", "IS_DINNER", "IS_DAWN"]
        row = row_data[feature_cols]
        
        pred_amt, pred_cnt = models[h].predict(row)[0]
        
        outs.append({
            "TA_YMD": yyyymmdd,
            "DONG": dong,
            "HOUR": h,
            "TEMP": float(row_data["TEMP"].iloc[0]),
            "RAIN": float(row_data["RAIN"].iloc[0]),
            "PRED_AMT": max(0, float(pred_amt)),  # 음수 방지
            "PRED_CNT": max(0, float(pred_cnt)),  # 음수 방지
        })
    
    return pd.DataFrame(outs)


# =========================
# H) 실행 예시
# =========================
if __name__ == "__main__":
    # 1) 학습 + 저장
    print("="*60)
    print("모델 학습 시작")
    print("="*60)
    metrics_df = train_and_save_models()
    print("\n최종 성능 지표:")
    print(metrics_df.to_string(index=False))
    
    # 2) 예측 예시
    if not SERVICE_KEY:
        print("\n[경고] 환경변수 RAIN_ID가 비었습니다. 예측을 하려면 SERVICE_KEY를 넣으세요.\n")
    else:
        try:
            yyyymmdd = "20251230"
            dong = "매교동"
            print(f"\n{'='*60}")
            print(f"예측 예시: {dong}, {yyyymmdd}")
            print(f"{'='*60}")
            pred_df = predict_day(yyyymmdd, dong)
            print(pred_df.to_string(index=False))
            
            output_file = f"pred_{dong}_{yyyymmdd}.csv"
            pred_df.to_csv(output_file, index=False, encoding="utf-8-sig")
            print(f"\n예측 결과 저장: {output_file}")
        except Exception as e:
            print(f"\n예측 중 오류 발생: {e}")

모델 학습 시작


C:\Users\Admin\anaconda3\envs\ml-dl-nlp\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


원본 데이터: 443,523건
이상치 제거 후: 439,087건

HOUR 01 모델 학습 중...

[HOUR 01] 저장 완료 -> data/models_hourly_amt_cnt2\hour_01_amt_cnt.joblib
  AMT - MAE: 728,242.0 | R²: 0.8269 | RMSE: 1,600,828.6
  CNT - MAE: 12.98 | R²: 0.8214 | RMSE: 39.51

HOUR 02 모델 학습 중...

[HOUR 02] 저장 완료 -> data/models_hourly_amt_cnt2\hour_02_amt_cnt.joblib
  AMT - MAE: 244,750.7 | R²: 0.4233 | RMSE: 785,378.3
  CNT - MAE: 7.64 | R²: 0.8720 | RMSE: 14.40

HOUR 03 모델 학습 중...


ValueError: Input X contains infinity or a value too large for dtype('float64').

# 9. 이상치 제거 후 정님 모델

In [9]:
%%time
"""
시간대별 AMT/CNT 예측 (수원시 한식 동별 데이터) - 개선 버전
- 입력: 날짜(TA_YMD), DONG, (기상청 API로 가져온) 시간대별 TEMP/RAIN
- 출력: 시간대별 AMT, CNT

개선사항:
1. 피처 엔지니어링 강화 (온도 구간, 계절, 주말 등)
2. 하이퍼파라미터 최적화
3. 이상치 처리
4. R² 스코어 평가 추가
5. 모델별 최적화
"""

import os
import math
import joblib
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from dotenv import load_dotenv

# =========================
# A) 설정
# =========================
load_dotenv()
SERVICE_KEY = os.getenv('RAIN_ID')
DATA_CSV = "data/수원시 한식 동별 데이터백업.csv"
MODEL_DIR = "data/models_hourly_amt_cnt_improved"
os.makedirs(MODEL_DIR, exist_ok=True)

# ---- 기상청 단기예보(getVilageFcst) 엔드포인트 ----
KMA_VILAGE_URL = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
BASE_TIMES = ["2300", "2000", "1700", "1400", "1100", "0800", "0500", "0200"]

# HOUR(01~10) 구간 대표 시각
HOUR_SLOT_TO_REP_TIME = {
    1: "0300", 2: "0800", 3: "1000", 4: "1200", 5: "1400",
    6: "1600", 7: "1800", 8: "2000", 9: "2200", 10: "2300",
}

# ---- DONG -> (nx, ny) 격자 좌표 매핑 ----
DONG_GRID_CSV = "data/dong_grid.csv"
DONG_TO_GRID = {}


# =========================
# B) 유틸: 요일 계산 (1~7, 월=1 ... 일=7)
# =========================
def ymd_to_day_1to7(yyyymmdd: str) -> int:
    dt = datetime.strptime(str(yyyymmdd), "%Y%m%d")
    return dt.weekday() + 1


# =========================
# C) 피처 엔지니어링 함수 추가
# =========================
def create_features(df):
    """
    추가 피처 생성:
    - TEMP_BIN: 온도 구간 (분석에서 발견한 최적 구간 활용)
    - IS_WEEKEND: 주말 여부
    - SEASON: 계절
    - TEMP_SQUARED: 온도 제곱 (비선형 관계)
    - IS_HOT: 무더운 날씨 (27도 이상)
    - IS_COLD: 추운 날씨 (-7~1도)
    """
    df = df.copy()
    
    # 온도 구간 (분석 결과 기반)
    df['TEMP_BIN'] = pd.cut(df['TEMP'], 
                            bins=[-100, -7, 1, 10, 18, 27, 100],
                            labels=['very_cold', 'cold', 'cool', 'pleasant', 'warm', 'hot'])
    df['TEMP_BIN'] = df['TEMP_BIN'].astype(str)
    
    # 주말 여부 (토=6, 일=7)
    df['IS_WEEKEND'] = (df['DAY'] >= 6).astype(int)
    
    # 계절 (월 추출 필요)
    df['MONTH'] = df['TA_YMD'].astype(str).str[4:6].astype(int)
    df['SEASON'] = df['MONTH'].apply(lambda m: 
        'spring' if 3 <= m <= 5 else
        'summer' if 6 <= m <= 8 else
        'fall' if 9 <= m <= 11 else 'winter'
    )
    
    # 온도 제곱 (역U자 패턴 반영)
    df['TEMP_SQUARED'] = df['TEMP'] ** 2
    
    # 특정 온도 구간 플래그
    df['IS_HOT'] = (df['TEMP'] >= 27).astype(int)
    df['IS_COLD'] = ((df['TEMP'] >= -7) & (df['TEMP'] <= 1)).astype(int)
    df['IS_OPTIMAL'] = ((df['TEMP'] >= 10) & (df['TEMP'] <= 14)).astype(int)
    
    # 강수량 로그 변환 - 음수 값 0으로 처리
    df['RAIN_CLIPPED'] = df['RAIN'].clip(lower=0)  # 음수를 0으로
    df['RAIN_LOG'] = np.log1p(df['RAIN_CLIPPED'])
    
    # 시간대 특성
    df['IS_LUNCH'] = (df['HOUR'] == 4).astype(int)  # 점심시간대
    df['IS_DINNER'] = (df['HOUR'].isin([7, 8])).astype(int)  # 저녁시간대
    df['IS_DAWN'] = (df['HOUR'] == 1).astype(int)  # 새벽시간대
    
    return df


def remove_outliers(df, target_col, threshold_percentile=99):
    """이상치 제거"""
    threshold = df[target_col].quantile(threshold_percentile / 100)
    return df[df[target_col] <= threshold]


# =========================
# D) 기상청 API 함수들 (원본 유지)
# =========================
def pick_base_datetime_kst(now_kst: datetime) -> tuple[str, str]:
    today = now_kst.strftime("%Y%m%d")
    hhmm = now_kst.strftime("%H%M")
    for bt in BASE_TIMES:
        if hhmm >= bt:
            return today, bt
    yday = (now_kst - timedelta(days=1)).strftime("%Y%m%d")
    return yday, "2300"


def fetch_vilage_fcst_items(service_key: str, base_date: str, base_time: str, nx: int, ny: int,
                            num_of_rows: int = 2000) -> list[dict]:
    params = {
        "serviceKey": service_key,
        "pageNo": 1,
        "numOfRows": num_of_rows,
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": nx,
        "ny": ny,
    }
    r = requests.get(KMA_VILAGE_URL, params=params, timeout=20)
    r.raise_for_status()
    data = r.json()
    items = data["response"]["body"]["items"]["item"]
    return items


def parse_pcp_to_float(pcp_val) -> float:
    if pcp_val is None:
        return 0.0
    s = str(pcp_val).strip()
    if s in ("강수없음", "없음", ""):
        return 0.0
    s = s.replace("mm", "").replace("미만", "").strip()
    try:
        return float(s)
    except:
        return 0.0


def get_temp_rain_for_datetime(service_key: str, dong: str, target_yyyymmdd: str, target_hhmm: str) -> tuple[float, float]:
    if not DONG_TO_GRID:
        if os.path.exists(DONG_GRID_CSV):
            g = pd.read_csv(DONG_GRID_CSV)
            for _, row in g.iterrows():
                DONG_TO_GRID[str(row["DONG"])] = (int(row["nx"]), int(row["ny"]))
        else:
            raise ValueError(f"[필수] {DONG_GRID_CSV}를 만들거나 DONG_TO_GRID dict를 채워주세요.")

    if dong not in DONG_TO_GRID:
        raise ValueError(f"DONG_TO_GRID에 '{dong}'의 nx/ny가 없습니다.")

    nx, ny = DONG_TO_GRID[dong]
    now_kst = datetime.now(timezone(timedelta(hours=9)))
    base_date, base_time = pick_base_datetime_kst(now_kst)
    items = fetch_vilage_fcst_items(service_key, base_date, base_time, nx, ny)

    tmp = None
    pcp = None
    for it in items:
        if it.get("fcstDate") == target_yyyymmdd and it.get("fcstTime") == target_hhmm:
            cat = it.get("category")
            if cat == "TMP":
                tmp = float(it.get("fcstValue"))
            elif cat == "PCP":
                pcp = parse_pcp_to_float(it.get("fcstValue"))

    if tmp is None:
        tmp = 0.0
    if pcp is None:
        pcp = 0.0
    return tmp, pcp


def get_hourly_weather_features(service_key: str, dong: str, yyyymmdd: str) -> pd.DataFrame:
    day_1to7 = ymd_to_day_1to7(yyyymmdd)
    rows = []
    for hour_slot in range(1, 11):
        rep_time = HOUR_SLOT_TO_REP_TIME[hour_slot]
        temp, rain = get_temp_rain_for_datetime(service_key, dong, yyyymmdd, rep_time)
        rows.append({
            "TA_YMD": int(yyyymmdd),
            "DONG": dong,
            "HOUR": hour_slot,
            "DAY": day_1to7,
            "TEMP": float(temp),
            "RAIN": float(rain),
        })
    df = pd.DataFrame(rows)
    return create_features(df)


# =========================
# E) 개선된 전처리 파이프라인
# =========================
def build_preprocess():
    """개선된 전처리 - 더 많은 피처 활용"""
    cat_cols = ["DONG", "TEMP_BIN", "SEASON"]
    num_cols = ["DAY", "TEMP", "RAIN", "TEMP_SQUARED", "RAIN_LOG",
                "IS_WEEKEND", "IS_HOT", "IS_COLD", "IS_OPTIMAL",
                "IS_LUNCH", "IS_DINNER", "IS_DAWN"]
    
    pre = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
            ("num", StandardScaler(), num_cols),  # 스케일링 추가
        ],
        remainder="drop",
    )
    return pre


# =========================
# F) 개선된 모델 학습
# =========================
def train_and_save_models(random_state: int = 42):
    df = pd.read_csv(DATA_CSV)
    
    # 타입 정리
    df["DONG"] = df["DONG"].astype(str)
    df["HOUR"] = df["HOUR"].astype(int)
    df["DAY"] = df["DAY"].astype(int)
    df["TEMP"] = df["TEMP"].astype(float)
    df["RAIN"] = df["RAIN"].astype(float)
    
    # 피처 생성
    df = create_features(df)
    
    # 무한대 및 NaN 제거
    print(f"피처 생성 후: {len(df):,}건")
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna()
    print(f"무한대/NaN 제거 후: {len(df):,}건")
    
    # 이상치 제거 (AMT 기준 상위 1%)
    df = remove_outliers(df, "AMT", threshold_percentile=99)
    print(f"이상치 제거 후: {len(df):,}건")
    
    y_cols = ["AMT", "CNT"]
    metrics = []

    for h in range(1, 11):
        print(f"\n{'='*60}")
        print(f"HOUR {h:02d} 모델 학습 중...")
        print(f"{'='*60}")
        
        sub = df[df["HOUR"] == h].copy()
        sub = sub.dropna(subset=["DONG", "DAY", "TEMP", "RAIN", "AMT", "CNT"])
        
        # 피처 선택
        feature_cols = ["DONG", "DAY", "TEMP", "RAIN", "TEMP_BIN", "SEASON",
                       "TEMP_SQUARED", "RAIN_LOG", "IS_WEEKEND", "IS_HOT", 
                       "IS_COLD", "IS_OPTIMAL", "IS_LUNCH", "IS_DINNER", "IS_DAWN"]
        
        X = sub[feature_cols]
        y = sub[y_cols]
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state, shuffle=True
        )
        
        pre = build_preprocess()
        
        if h == 1:
            # HOUR=1: 개선된 MLPRegressor
            base_model = MLPRegressor(
                hidden_layer_sizes=(256, 128, 64),  # 더 깊은 네트워크
                activation="relu",
                solver="adam",
                alpha=1e-4,
                batch_size=128,  # 배치 크기 감소
                learning_rate="adaptive",  # 적응형 학습률
                learning_rate_init=5e-4,
                max_iter=500,  # 더 많은 에포크
                random_state=42,
                early_stopping=True,
                validation_fraction=0.1,
                n_iter_no_change=20,
                tol=1e-5,
                verbose=False,
            )
        else:
            # HOUR=2~10: 최적화된 XGBRegressor
            base_model = XGBRegressor(
                n_estimators=800,  # 트리 개수
                learning_rate=0.04,  # 학습률
                max_depth=7,  # 깊이
                min_child_weight=3,
                subsample=0.8,
                colsample_bytree=0.8,
                reg_alpha=0.1,  # L1 정규화
                reg_lambda=1.5,  # L2 정규화
                gamma=0.1,  # 최소 손실 감소
                objective="reg:squarederror",
                random_state=random_state,
                tree_method="hist",
                n_jobs=-1,
            )
        
        model = MultiOutputRegressor(base_model)
        
        pipe = Pipeline([
            ("pre", pre),
            ("model", model),
        ])
        
        # 모델 학습
        pipe.fit(X_train, y_train)
        
        # 예측 및 평가
        pred = pipe.predict(X_test)
        
        # AMT 평가
        mae_amt = mean_absolute_error(y_test["AMT"].values, pred[:, 0])
        r2_amt = r2_score(y_test["AMT"].values, pred[:, 0])
        rmse_amt = np.sqrt(mean_squared_error(y_test["AMT"].values, pred[:, 0]))
        
        # CNT 평가
        mae_cnt = mean_absolute_error(y_test["CNT"].values, pred[:, 1])
        r2_cnt = r2_score(y_test["CNT"].values, pred[:, 1])
        rmse_cnt = np.sqrt(mean_squared_error(y_test["CNT"].values, pred[:, 1]))
        
        # 모델 저장
        model_path = os.path.join(MODEL_DIR, f"hour_{h:02d}_amt_cnt.joblib")
        joblib.dump(pipe, model_path)
        
        metrics.append({
            "HOUR": h,
            "MAE_AMT": mae_amt,
            "R2_AMT": r2_amt,
            "RMSE_AMT": rmse_amt,
            "MAE_CNT": mae_cnt,
            "R2_CNT": r2_cnt,
            "RMSE_CNT": rmse_cnt,
            "model_path": model_path,
            "model_type": "MLP(Deep)" if h == 1 else "XGB",
            "n_train": len(X_train),
            "n_test": len(X_test),
        })
        
        print(f"\n[HOUR {h:02d}] 저장 완료 -> {model_path}")
        print(f"  AMT - MAE: {mae_amt:,.1f} | R²: {r2_amt:.4f} | RMSE: {rmse_amt:,.1f}")
        print(f"  CNT - MAE: {mae_cnt:,.2f} | R²: {r2_cnt:.4f} | RMSE: {rmse_cnt:,.2f}")
    
    metrics_df = pd.DataFrame(metrics).sort_values("HOUR")
    metrics_df.to_csv(os.path.join(MODEL_DIR, "metrics_summary.csv"), index=False, encoding="utf-8-sig")
    
    print("\n" + "="*60)
    print("전체 모델 성능 요약")
    print("="*60)
    print(f"\nAMT 평균 성능:")
    print(f"  - MAE: {metrics_df['MAE_AMT'].mean():,.1f}")
    print(f"  - R²: {metrics_df['R2_AMT'].mean():.4f}")
    print(f"\nCNT 평균 성능:")
    print(f"  - MAE: {metrics_df['MAE_CNT'].mean():.2f}")
    print(f"  - R²: {metrics_df['R2_CNT'].mean():.4f}")
    print("\n=== metrics_summary.csv 저장 완료 ===\n")
    
    return metrics_df


# =========================
# G) 예측 함수 (원본 유지하되 피처 추가)
# =========================
def load_models():
    models = {}
    for h in range(1, 11):
        p = os.path.join(MODEL_DIR, f"hour_{h:02d}_amt_cnt.joblib")
        if not os.path.exists(p):
            raise FileNotFoundError(f"모델 파일이 없습니다: {p}")
        models[h] = joblib.load(p)
    return models


def predict_day(yyyymmdd: str, dong: str) -> pd.DataFrame:
    # 기상청에서 시간대별 TEMP/RAIN 구성 (피처 포함)
    feat_df = get_hourly_weather_features(SERVICE_KEY, dong, yyyymmdd)
    
    models = load_models()
    
    outs = []
    for h in range(1, 11):
        row_data = feat_df[feat_df["HOUR"] == h]
        
        # 필요한 피처만 선택
        feature_cols = ["DONG", "DAY", "TEMP", "RAIN", "TEMP_BIN", "SEASON",
                       "TEMP_SQUARED", "RAIN_LOG", "IS_WEEKEND", "IS_HOT", 
                       "IS_COLD", "IS_OPTIMAL", "IS_LUNCH", "IS_DINNER", "IS_DAWN"]
        row = row_data[feature_cols]
        
        pred_amt, pred_cnt = models[h].predict(row)[0]
        
        outs.append({
            "TA_YMD": yyyymmdd,
            "DONG": dong,
            "HOUR": h,
            "TEMP": float(row_data["TEMP"].iloc[0]),
            "RAIN": float(row_data["RAIN"].iloc[0]),
            "PRED_AMT": max(0, float(pred_amt)),  # 음수 방지
            "PRED_CNT": max(0, float(pred_cnt)),  # 음수 방지
        })
    
    return pd.DataFrame(outs)

모델 학습 시작
피처 생성 후: 443,523건
무한대/NaN 제거 후: 443,523건
이상치 제거 후: 439,087건

HOUR 01 모델 학습 중...

[HOUR 01] 저장 완료 -> data/models_hourly_amt_cnt_improved\hour_01_amt_cnt.joblib
  AMT - MAE: 733,359.3 | R²: 0.8268 | RMSE: 1,600,950.1
  CNT - MAE: 12.98 | R²: 0.8215 | RMSE: 39.50

HOUR 02 모델 학습 중...

[HOUR 02] 저장 완료 -> data/models_hourly_amt_cnt_improved\hour_02_amt_cnt.joblib
  AMT - MAE: 244,781.1 | R²: 0.4236 | RMSE: 785,191.4
  CNT - MAE: 7.63 | R²: 0.8720 | RMSE: 14.39

HOUR 03 모델 학습 중...

[HOUR 03] 저장 완료 -> data/models_hourly_amt_cnt_improved\hour_03_amt_cnt.joblib
  AMT - MAE: 584,866.4 | R²: 0.4755 | RMSE: 1,487,117.6
  CNT - MAE: 14.82 | R²: 0.8495 | RMSE: 28.28

HOUR 04 모델 학습 중...

[HOUR 04] 저장 완료 -> data/models_hourly_amt_cnt_improved\hour_04_amt_cnt.joblib
  AMT - MAE: 1,293,354.1 | R²: 0.8980 | RMSE: 2,551,237.1
  CNT - MAE: 39.91 | R²: 0.9280 | RMSE: 75.61

HOUR 05 모델 학습 중...

[HOUR 05] 저장 완료 -> data/models_hourly_amt_cnt_improved\hour_05_amt_cnt.joblib
  AMT - MAE: 1,408,571.2 | R²

In [ ]:
# =========================
# G) 실행 예시
# =========================
if __name__ == "__main__":
    # 1) 학습 + 저장
    print("="*60)
    print("모델 학습 시작")
    print("="*60)
    metrics_df = train_and_save_models()
    print("\n최종 성능 지표:")
    print(metrics_df.to_string(index=False))

    # 2) 예측 예시
    if not SERVICE_KEY:
        print("\n[경고] 환경변수 RAIN_ID가 비었습니다. 예측을 하려면 SERVICE_KEY를 넣으세요.\n")
    else:
        try:
            yyyymmdd = "20251230"
            dong = "매교동"
            print(f"\n{'='*60}")
            print(f"예측 예시: {dong}, {yyyymmdd}")
            print(f"{'='*60}")
            pred_df = predict_day(yyyymmdd, dong)
            print(pred_df.to_string(index=False))
            
            output_file = f"pred_{dong}_{yyyymmdd}.csv"
            pred_df.to_csv(output_file, index=False, encoding="utf-8-sig")
            print(f"\n예측 결과 저장: {output_file}")
        except Exception as e:
            print(f"\n예측 중 오류 발생: {e}")

In [13]:
%%time
"""
수원시 한식 동별 데이터 - 시간대별 AMT/CNT 예측 (MAE 개선 반영)
✅ Target Encoding (AMT 기준, 누수 방지: train 기준 mapping)
✅ 시계열 Train/Test split (TA_YMD 정렬 후 마지막 20% 테스트)
✅ HOUR별 XGB 파라미터 분리
✅ CNT → AMT 2-Stage (AMT 입력에 PRED_CNT 포함, train에선 OOF PRED_CNT로 누수 방지)
✅ 기상청 API 429 방지: "하루 예측 시 API 1회 호출" + 429 backoff 재시도

[학습]
- train_and_save_models() 실행 → MODEL_DIR에 hour_01_bundle.joblib ~ hour_10_bundle.joblib 저장

[예측]
- predict_day("YYYYMMDD", "행궁동") → 시간대별 PRED_AMT, PRED_CNT 반환

필수 파일
- DATA_CSV: data/수원시 한식 동별 데이터백업.csv  (TA_YMD,DONG,HOUR,DAY,TEMP,RAIN,AMT,CNT 포함)
- DONG_GRID_CSV: data/dong_grid.csv  (DONG,nx,ny 포함)
- .env에 RAIN_ID(서비스키) 또는 환경변수 RAIN_ID 설정
"""

import os
import time
import joblib
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import KFold

from xgboost import XGBRegressor
from dotenv import load_dotenv


# =========================
# A) 설정
# =========================
load_dotenv()
SERVICE_KEY = os.getenv("RAIN_ID")  # 기상청 단기예보 서비스키 (필수)

DATA_CSV = "data/수원시 한식 동별 데이터백업.csv"
MODEL_DIR = "data/models_hourly_amt_cnt_2stage_te"
os.makedirs(MODEL_DIR, exist_ok=True)

# ---- 기상청 단기예보(getVilageFcst) 엔드포인트 ----
KMA_VILAGE_URL = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
BASE_TIMES = ["2300", "2000", "1700", "1400", "1100", "0800", "0500", "0200"]

# HOUR(01~10) 구간 대표 시각
HOUR_SLOT_TO_REP_TIME = {
    1: "0300", 2: "0800", 3: "1000", 4: "1200", 5: "1400",
    6: "1600", 7: "1800", 8: "2000", 9: "2200", 10: "2300",
}

# ---- DONG -> (nx, ny) 격자 좌표 매핑 ----
DONG_GRID_CSV = "data/dong_grid.csv"
DONG_TO_GRID = {}  # lazy load


# =========================
# B) 유틸: 요일 계산 (1~7, 월=1 ... 일=7)
# =========================
def ymd_to_day_1to7(yyyymmdd: str) -> int:
    dt = datetime.strptime(str(yyyymmdd), "%Y%m%d")
    return dt.weekday() + 1


# =========================
# C) 피처 엔지니어링
# =========================
def create_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    파생 피처:
    - TEMP_BIN, IS_WEEKEND, SEASON
    - TEMP_SQUARED, RAIN_LOG
    - IS_HOT/IS_COLD/IS_OPTIMAL
    - 시간대 플래그(IS_LUNCH/IS_DINNER/IS_DAWN)
    """
    df = df.copy()

    df["TEMP_BIN"] = pd.cut(
        df["TEMP"],
        bins=[-100, -7, 1, 10, 18, 27, 100],
        labels=["very_cold", "cold", "cool", "pleasant", "warm", "hot"],
    ).astype(str)

    df["IS_WEEKEND"] = (df["DAY"] >= 6).astype(int)

    df["MONTH"] = df["TA_YMD"].astype(str).str[4:6].astype(int)
    df["SEASON"] = df["MONTH"].apply(lambda m:
        "spring" if 3 <= m <= 5 else
        "summer" if 6 <= m <= 8 else
        "fall"   if 9 <= m <= 11 else
        "winter"
    )

    df["TEMP_SQUARED"] = df["TEMP"] ** 2

    df["IS_HOT"] = (df["TEMP"] >= 27).astype(int)
    df["IS_COLD"] = ((df["TEMP"] >= -7) & (df["TEMP"] <= 1)).astype(int)
    df["IS_OPTIMAL"] = ((df["TEMP"] >= 10) & (df["TEMP"] <= 14)).astype(int)

    df["RAIN_CLIPPED"] = df["RAIN"].clip(lower=0)
    df["RAIN_LOG"] = np.log1p(df["RAIN_CLIPPED"])

    df["IS_LUNCH"] = (df["HOUR"] == 4).astype(int)
    df["IS_DINNER"] = (df["HOUR"].isin([7, 8])).astype(int)
    df["IS_DAWN"] = (df["HOUR"] == 1).astype(int)

    return df


def remove_outliers(df: pd.DataFrame, target_col: str, threshold_percentile: int = 99) -> pd.DataFrame:
    thr = df[target_col].quantile(threshold_percentile / 100)
    return df[df[target_col] <= thr]


# =========================
# D) 기상청 API (429 방지: 1회 호출 + backoff)
# =========================
def pick_base_datetime_kst(now_kst: datetime) -> tuple[str, str]:
    today = now_kst.strftime("%Y%m%d")
    hhmm = now_kst.strftime("%H%M")
    for bt in BASE_TIMES:
        if hhmm >= bt:
            return today, bt
    yday = (now_kst - timedelta(days=1)).strftime("%Y%m%d")
    return yday, "2300"


def fetch_vilage_fcst_items(
    service_key: str, base_date: str, base_time: str, nx: int, ny: int,
    num_of_rows: int = 2000, max_retries: int = 6
) -> list[dict]:
    params = {
        "serviceKey": service_key,
        "pageNo": 1,
        "numOfRows": num_of_rows,
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": nx,
        "ny": ny,
    }

    wait = 1.0
    last_status = None
    for _ in range(max_retries):
        r = requests.get(KMA_VILAGE_URL, params=params, timeout=20)
        last_status = r.status_code

        if r.status_code == 429:
            time.sleep(wait)
            wait = min(wait * 2, 30.0)
            continue

        r.raise_for_status()
        data = r.json()
        return data["response"]["body"]["items"]["item"]

    raise requests.HTTPError(f"HTTP {last_status}: Too Many Requests(429) - 재시도 {max_retries}회 실패")


def parse_pcp_to_float(pcp_val) -> float:
    if pcp_val is None:
        return 0.0
    s = str(pcp_val).strip()
    if s in ("강수없음", "없음", ""):
        return 0.0
    s = s.replace("mm", "").replace("미만", "").strip()
    try:
        return float(s)
    except Exception:
        return 0.0


def _load_dong_grid_if_needed():
    global DONG_TO_GRID
    if DONG_TO_GRID:
        return
    if not os.path.exists(DONG_GRID_CSV):
        raise ValueError(f"[필수] {DONG_GRID_CSV} 파일이 없습니다. (DONG,nx,ny 필요)")
    g = pd.read_csv(DONG_GRID_CSV)
    if not {"DONG", "nx", "ny"}.issubset(set(g.columns)):
        raise ValueError(f"[오류] {DONG_GRID_CSV} 컬럼이 DONG,nx,ny를 포함해야 합니다. 현재={list(g.columns)}")
    for _, row in g.iterrows():
        DONG_TO_GRID[str(row["DONG"])] = (int(row["nx"]), int(row["ny"]))


def get_vilage_items_once_for_day(service_key: str, dong: str) -> dict:
    """
    예보 items를 한 번만 받아서 (fcstDate, fcstTime) -> {TMP, PCP} 형태 dict로 캐시화
    """
    _load_dong_grid_if_needed()

    if dong not in DONG_TO_GRID:
        raise ValueError(f"DONG_GRID에 '{dong}'의 nx/ny가 없습니다.")

    nx, ny = DONG_TO_GRID[dong]
    now_kst = datetime.now(timezone(timedelta(hours=9)))
    base_date, base_time = pick_base_datetime_kst(now_kst)

    items = fetch_vilage_fcst_items(service_key, base_date, base_time, nx, ny)

    d = {}
    for it in items:
        fcstDate = it.get("fcstDate")
        fcstTime = it.get("fcstTime")
        cat = it.get("category")
        val = it.get("fcstValue")
        key = (fcstDate, fcstTime)
        if key not in d:
            d[key] = {}
        d[key][cat] = val
    return d


def get_hourly_weather_features(service_key: str, dong: str, yyyymmdd: str) -> pd.DataFrame:
    """
    ✅ 예측 1회당 API 1회 호출
    """
    if not service_key:
        raise ValueError("SERVICE_KEY(RAIN_ID)가 비어있습니다. .env 또는 환경변수에 RAIN_ID를 설정하세요.")

    day_1to7 = ymd_to_day_1to7(yyyymmdd)
    items_dict = get_vilage_items_once_for_day(service_key, dong)

    rows = []
    for hour_slot in range(1, 11):
        rep_time = HOUR_SLOT_TO_REP_TIME[hour_slot]
        key = (yyyymmdd, rep_time)

        tmp = 0.0
        pcp = 0.0
        if key in items_dict:
            if "TMP" in items_dict[key]:
                try:
                    tmp = float(items_dict[key]["TMP"])
                except Exception:
                    tmp = 0.0
            if "PCP" in items_dict[key]:
                pcp = parse_pcp_to_float(items_dict[key]["PCP"])

        rows.append({
            "TA_YMD": int(yyyymmdd),
            "DONG": dong,
            "HOUR": hour_slot,
            "DAY": day_1to7,
            "TEMP": float(tmp),
            "RAIN": float(pcp),
        })

    df = pd.DataFrame(rows)
    return create_features(df)


# =========================
# E) Target Encoding (AMT 기준, 누수 방지)
# =========================
class DongTargetEncoder:
    """
    DONG -> AMT mean encoding
    fit: train 데이터로 dong별 평균 계산
    transform: 매핑 적용, 없으면 global mean
    """
    def __init__(self, target_col: str = "AMT", dong_col: str = "DONG", out_col: str = "DONG_TE"):
        self.target_col = target_col
        self.dong_col = dong_col
        self.out_col = out_col
        self.mapping_ = {}
        self.global_mean_ = 0.0

    def fit(self, train_df: pd.DataFrame):
        if self.dong_col not in train_df.columns or self.target_col not in train_df.columns:
            raise ValueError(f"TE fit에 {self.dong_col},{self.target_col} 컬럼이 필요합니다.")
        self.global_mean_ = float(train_df[self.target_col].mean())
        self.mapping_ = train_df.groupby(self.dong_col)[self.target_col].mean().to_dict()
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X2 = X.copy()
        X2[self.out_col] = X2[self.dong_col].map(self.mapping_).fillna(self.global_mean_)
        return X2


# =========================
# F) 전처리
# - DONG은 OneHot 하지 않고 DONG_TE 사용
# =========================
def build_preprocess(use_pred_cnt: bool) -> ColumnTransformer:
    cat_cols = ["TEMP_BIN", "SEASON"]
    num_cols = [
        "DAY", "TEMP", "RAIN",
        "TEMP_SQUARED", "RAIN_LOG",
        "IS_WEEKEND", "IS_HOT", "IS_COLD", "IS_OPTIMAL",
        "IS_LUNCH", "IS_DINNER", "IS_DAWN",
        "DONG_TE",
    ]
    if use_pred_cnt:
        num_cols += ["PRED_CNT"]

    pre = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
            ("num", StandardScaler(), num_cols),
        ],
        remainder="drop",
    )
    return pre


# =========================
# G) HOUR별 XGB 파라미터 분리
# =========================
def get_xgb_params_cnt(h: int, random_state: int = 42) -> dict:
    # CNT는 상대적으로 안정적: 과적합 방지 쪽
    if h in [4, 5, 7, 8]:  # 피크
        return dict(
            n_estimators=900, learning_rate=0.04, max_depth=7,
            subsample=0.85, colsample_bytree=0.85,
            min_child_weight=3, reg_alpha=0.0, reg_lambda=1.2, gamma=0.0,
            objective="reg:squarederror",
            random_state=random_state, tree_method="hist", n_jobs=-1
        )
    return dict(
        n_estimators=700, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8,
        min_child_weight=4, reg_alpha=0.0, reg_lambda=1.3, gamma=0.0,
        objective="reg:squarederror",
        random_state=random_state, tree_method="hist", n_jobs=-1
    )


def get_xgb_params_amt(h: int, random_state: int = 42) -> dict:
    # AMT는 변동이 큼: 표현력 조금 더
    if h in [4, 5]:  # 점심
        return dict(
            n_estimators=1200, learning_rate=0.03, max_depth=8,
            subsample=0.85, colsample_bytree=0.85,
            min_child_weight=3, reg_alpha=0.05, reg_lambda=1.2, gamma=0.05,
            objective="reg:squarederror",
            random_state=random_state, tree_method="hist", n_jobs=-1
        )
    if h in [7, 8]:  # 저녁
        return dict(
            n_estimators=1400, learning_rate=0.028, max_depth=9,
            subsample=0.85, colsample_bytree=0.85,
            min_child_weight=3, reg_alpha=0.05, reg_lambda=1.3, gamma=0.08,
            objective="reg:squarederror",
            random_state=random_state, tree_method="hist", n_jobs=-1
        )
    return dict(
        n_estimators=900, learning_rate=0.04, max_depth=7,
        subsample=0.8, colsample_bytree=0.8,
        min_child_weight=4, reg_alpha=0.05, reg_lambda=1.5, gamma=0.05,
        objective="reg:squarederror",
        random_state=random_state, tree_method="hist", n_jobs=-1
    )


# =========================
# H) 학습/저장 (2-stage + TE + 시계열 split)
# =========================
def train_and_save_models(
    random_state: int = 42,
    test_ratio: float = 0.2,
    oof_folds: int = 5,
    outlier_percentile: int = 99
) -> pd.DataFrame:
    if not os.path.exists(DATA_CSV):
        raise FileNotFoundError(f"데이터 파일이 없습니다: {DATA_CSV}")

    df = pd.read_csv(DATA_CSV)

    required_cols = {"TA_YMD","DONG","HOUR","DAY","TEMP","RAIN","AMT","CNT"}
    if not required_cols.issubset(set(df.columns)):
        raise ValueError(f"DATA_CSV에 필요한 컬럼이 부족합니다. 필요={sorted(required_cols)} / 현재={list(df.columns)}")

    # 타입 정리
    df["TA_YMD"] = df["TA_YMD"].astype(int)
    df["DONG"] = df["DONG"].astype(str)
    df["HOUR"] = df["HOUR"].astype(int)
    df["DAY"] = df["DAY"].astype(int)
    df["TEMP"] = df["TEMP"].astype(float)
    df["RAIN"] = df["RAIN"].astype(float)
    df["AMT"] = df["AMT"].astype(float)
    df["CNT"] = df["CNT"].astype(float)

    # 피처 생성
    df = create_features(df)

    # 무한/NaN 제거
    df = df.replace([np.inf, -np.inf], np.nan).dropna()

    # 이상치 제거 (AMT 기준 상위 1% 등)
    df = remove_outliers(df, "AMT", threshold_percentile=outlier_percentile)

    feature_base = [
        "TA_YMD", "DONG", "DAY", "TEMP", "RAIN", "TEMP_BIN", "SEASON",
        "TEMP_SQUARED", "RAIN_LOG", "IS_WEEKEND", "IS_HOT", "IS_COLD",
        "IS_OPTIMAL", "IS_LUNCH", "IS_DINNER", "IS_DAWN"
    ]

    metrics = []

    for h in range(1, 11):
        print("\n" + "="*60)
        print(f"HOUR {h:02d} 학습")
        print("="*60)

        sub = df[df["HOUR"] == h].copy()
        sub = sub.sort_values("TA_YMD")

        if len(sub) < 200:
            print(f"[경고] HOUR {h:02d} 데이터가 너무 적습니다: n={len(sub)} (그래도 진행)")
        if len(sub) < (oof_folds + 2):
            raise ValueError(f"HOUR {h:02d} 데이터가 너무 적어 OOF fold 불가: n={len(sub)}")

        # ✅ 시계열 split
        split = int(len(sub) * (1 - test_ratio))
        train_df = sub.iloc[:split].copy()
        test_df  = sub.iloc[split:].copy()

        # ✅ Target Encoding (AMT 기준, train만으로 fit)
        te = DongTargetEncoder(target_col="AMT", dong_col="DONG", out_col="DONG_TE").fit(train_df)

        train_X = te.transform(train_df[feature_base].copy())
        test_X  = te.transform(test_df[feature_base].copy())

        y_train_cnt = train_df["CNT"].values
        y_test_cnt  = test_df["CNT"].values
        y_train_amt = train_df["AMT"].values
        y_test_amt  = test_df["AMT"].values

        # -------- (1) CNT 모델 --------
        pre_cnt = build_preprocess(use_pred_cnt=False)
        pipe_cnt = Pipeline([
            ("pre", pre_cnt),
            ("model", XGBRegressor(**get_xgb_params_cnt(h, random_state=random_state))),
        ])

        # -------- (2) AMT 입력용 OOF PRED_CNT (누수 방지) --------
        # 시계열 느낌 유지: shuffle=False
        kf = KFold(n_splits=oof_folds, shuffle=False)
        train_X_reset = train_X.reset_index(drop=True)

        oof_pred_cnt = np.zeros(len(train_X_reset), dtype=float)

        for tr_idx, va_idx in kf.split(train_X_reset):
            X_tr = train_X_reset.iloc[tr_idx]
            X_va = train_X_reset.iloc[va_idx]
            y_tr = y_train_cnt[tr_idx]

            tmp_pipe = Pipeline([
                ("pre", pre_cnt),
                ("model", XGBRegressor(**get_xgb_params_cnt(h, random_state=random_state))),
            ])
            tmp_pipe.fit(X_tr, y_tr)
            oof_pred_cnt[va_idx] = tmp_pipe.predict(X_va)

        # CNT 최종 fit (전체 train)
        pipe_cnt.fit(train_X, y_train_cnt)

        # test용 pred_cnt
        pred_cnt_test = pipe_cnt.predict(test_X)

        # -------- (3) AMT 모델 (PRED_CNT 포함) --------
        train_X_amt = train_X.copy()
        test_X_amt  = test_X.copy()
        train_X_amt["PRED_CNT"] = oof_pred_cnt
        test_X_amt["PRED_CNT"] = pred_cnt_test

        pre_amt = build_preprocess(use_pred_cnt=True)
        pipe_amt = Pipeline([
            ("pre", pre_amt),
            ("model", XGBRegressor(**get_xgb_params_amt(h, random_state=random_state))),
        ])
        pipe_amt.fit(train_X_amt, y_train_amt)

        # -------- (4) 평가 --------
        pred_cnt = pipe_cnt.predict(test_X)
        pred_amt = pipe_amt.predict(test_X_amt)

        mae_cnt = mean_absolute_error(y_test_cnt, pred_cnt)
        r2_cnt  = r2_score(y_test_cnt, pred_cnt)
        rmse_cnt = np.sqrt(mean_squared_error(y_test_cnt, pred_cnt))

        mae_amt = mean_absolute_error(y_test_amt, pred_amt)
        r2_amt  = r2_score(y_test_amt, pred_amt)
        rmse_amt = np.sqrt(mean_squared_error(y_test_amt, pred_amt))

        # -------- (5) 저장 --------
        bundle = {
            "hour": h,
            "te_mapping": te.mapping_,
            "te_global_mean": te.global_mean_,
            "feature_base": feature_base,
            "pipe_cnt": pipe_cnt,
            "pipe_amt": pipe_amt,
        }
        model_path = os.path.join(MODEL_DIR, f"hour_{h:02d}_bundle.joblib")
        joblib.dump(bundle, model_path)

        metrics.append({
            "HOUR": h,
            "MAE_AMT": mae_amt,
            "R2_AMT": r2_amt,
            "RMSE_AMT": rmse_amt,
            "MAE_CNT": mae_cnt,
            "R2_CNT": r2_cnt,
            "RMSE_CNT": rmse_cnt,
            "n_train": len(train_df),
            "n_test": len(test_df),
            "model_path": model_path,
        })

        print(f"[HOUR {h:02d}] 저장 완료 -> {model_path}")
        print(f"  CNT - MAE {mae_cnt:,.2f} | R2 {r2_cnt:.4f} | RMSE {rmse_cnt:,.2f}")
        print(f"  AMT - MAE {mae_amt:,.1f} | R2 {r2_amt:.4f} | RMSE {rmse_amt:,.1f}")

    metrics_df = pd.DataFrame(metrics).sort_values("HOUR")
    out_csv = os.path.join(MODEL_DIR, "metrics_summary.csv")
    metrics_df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"\n=== metrics_summary.csv 저장 완료: {out_csv} ===")
    return metrics_df


# =========================
# I) 로드/예측
# =========================
def load_bundles() -> dict:
    bundles = {}
    for h in range(1, 11):
        p = os.path.join(MODEL_DIR, f"hour_{h:02d}_bundle.joblib")
        if not os.path.exists(p):
            raise FileNotFoundError(
                f"모델 번들 파일이 없습니다: {p}\n"
                f"먼저 train_and_save_models()를 실행해서 모델을 저장하세요."
            )
        bundles[h] = joblib.load(p)
    return bundles


def apply_te_from_bundle(bundle: dict, X: pd.DataFrame) -> pd.DataFrame:
    mp = bundle["te_mapping"]
    gm = bundle["te_global_mean"]
    X2 = X.copy()
    X2["DONG_TE"] = X2["DONG"].map(mp).fillna(gm)
    return X2


def predict_day(yyyymmdd: str, dong: str) -> pd.DataFrame:
    # 1) 기상/피처 (✅ API 1회)
    feat_df = get_hourly_weather_features(SERVICE_KEY, dong, yyyymmdd)

    # 2) 모델 번들 로드
    bundles = load_bundles()

    outs = []
    for h in range(1, 11):
        row = feat_df[feat_df["HOUR"] == h].copy()
        bundle = bundles[h]
        feature_base = bundle["feature_base"]

        X = row[feature_base].copy()
        X["DONG"] = dong  # 안전

        # TE 적용
        X = apply_te_from_bundle(bundle, X)

        # CNT 예측
        pred_cnt = float(bundle["pipe_cnt"].predict(X)[0])

        # AMT 예측 (PRED_CNT 추가)
        X_amt = X.copy()
        X_amt["PRED_CNT"] = pred_cnt
        pred_amt = float(bundle["pipe_amt"].predict(X_amt)[0])

        outs.append({
            "TA_YMD": str(yyyymmdd),
            "DONG": dong,
            "HOUR": h,
            "TEMP": float(row["TEMP"].iloc[0]),
            "RAIN": float(row["RAIN"].iloc[0]),
            "PRED_CNT": int(max(0, round(pred_cnt))),
            "PRED_AMT": int(max(0, round(pred_amt))),
        })

    return pd.DataFrame(outs)


# =========================
# J) 실행 예시
# =========================
if __name__ == "__main__":
    # ✅ 1) 학습이 먼저입니다. (최초 1회)
    metrics = train_and_save_models()
    print(metrics)

    # ✅ 2) 학습 후 예측
#     df_pred = predict_day("20260102", "행궁동")
#     print(df_pred)



HOUR 01 학습
[HOUR 01] 저장 완료 -> data/models_hourly_amt_cnt_2stage_te\hour_01_bundle.joblib
  CNT - MAE 18.82 | R2 0.7004 | RMSE 68.10
  AMT - MAE 816,196.0 | R2 0.8010 | RMSE 1,785,111.3

HOUR 02 학습
[HOUR 02] 저장 완료 -> data/models_hourly_amt_cnt_2stage_te\hour_02_bundle.joblib
  CNT - MAE 8.92 | R2 0.8807 | RMSE 15.63
  AMT - MAE 288,859.0 | R2 0.3426 | RMSE 1,009,549.0

HOUR 03 학습
[HOUR 03] 저장 완료 -> data/models_hourly_amt_cnt_2stage_te\hour_03_bundle.joblib
  CNT - MAE 18.14 | R2 0.8473 | RMSE 31.59
  AMT - MAE 656,629.0 | R2 0.4790 | RMSE 1,764,263.3

HOUR 04 학습
[HOUR 04] 저장 완료 -> data/models_hourly_amt_cnt_2stage_te\hour_04_bundle.joblib
  CNT - MAE 49.45 | R2 0.9024 | RMSE 92.13
  AMT - MAE 1,387,803.1 | R2 0.8945 | RMSE 2,638,367.1

HOUR 05 학습
[HOUR 05] 저장 완료 -> data/models_hourly_amt_cnt_2stage_te\hour_05_bundle.joblib
  CNT - MAE 42.15 | R2 0.8766 | RMSE 83.23
  AMT - MAE 1,508,768.1 | R2 0.8754 | RMSE 2,755,842.7

HOUR 06 학습
[HOUR 06] 저장 완료 -> data/models_hourly_amt_cnt_2stage_te

In [16]:
if __name__ == "__main__":
    df_pred = predict_day("20260103", "세류1동")
    print(df_pred)

     TA_YMD  DONG  HOUR  TEMP  RAIN  PRED_CNT  PRED_AMT
0  20260103  세류1동     1  -8.0   0.0         7    250257
1  20260103  세류1동     2  -8.0   0.0         2         0
2  20260103  세류1동     3  -5.0   0.0         8    124071
3  20260103  세류1동     4  -1.0   0.0         0     29494
4  20260103  세류1동     5   1.0   0.0        13    206055
5  20260103  세류1동     6   2.0   0.0        10    287139
6  20260103  세류1동     7   0.0   0.0        31     25590
7  20260103  세류1동     8  -1.0   0.0        10    390020
8  20260103  세류1동     9  -1.0   0.0         0     80202
9  20260103  세류1동    10   0.0   0.0         1    152991


In [ ]:
if __name__ == "__main__":
    df_pred = predict_day("20260103", "매산동")
    print(df_pred)

     TA_YMD DONG  HOUR  TEMP  RAIN  PRED_CNT  PRED_AMT
0  20260103  매산동     1  -8.0   0.0       433   9834727
1  20260103  매산동     2  -8.0   0.0       210   1108525
2  20260103  매산동     3  -5.0   0.0       425   5161708
3  20260103  매산동     4  -1.0   0.0       845  19481378
4  20260103  매산동     5   1.0   0.0       915  36747444
5  20260103  매산동     6   2.0   0.0       768  38014476
6  20260103  매산동     7   0.0   0.0       872  27775718
7  20260103  매산동     8  -1.0   0.0       839  34793352
8  20260103  매산동     9  -1.0   0.0       893  24592880
9  20260103  매산동    10   0.0   0.0       399   7321910
